## Helper

### Imports

In [26]:
from manim import *
import numpy as np
from manim_slides import Slide

### Deadish

#### (commented out) Add LaTeX to Path

In [27]:
# import os 
# print(os.environ["PATH"])
# os.environ["PATH"] = os.environ["PATH"].replace(r":/mnt/c/Users/irona/AppData/Local/Programs/MiKTeX/miktex/bin/x64/", "")
# os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2025/bin/x86_64-linux'
# print(os.environ["PATH"])

# If you're having trouble with "latex not found" kind of issues, just close and reopen vscode (idk why it works, but it does)


#### Tex Preamble (only used in qcircuit slide)

In [28]:
# Preamble
preamble_path = "/home/ronakr/Documents/inplaceoracles/Inverting-a-Permutation-with-an-In-Place-Oracle/preamble.tex"
with open(preamble_path, 'r') as file:
    preamble = file.read()
myTemplate = TexTemplate()
myTemplate.add_to_preamble(preamble)
# myTemplate.add_to_preamble(r"\usepackage{ragged2e}")
# 
# myTemplate.add_to_preamble(rf"\geometry{{reset, legalpaper, textwidth={DOCUMENT_TEXT_WIDTH}in}}")

TexTemplate(_body='', tex_compiler='latex', description='', output_format='.dvi', documentclass='\\documentclass[preview]{standalone}', preamble='\\usepackage[english]{babel}\n\\usepackage{amsmath}\n\\usepackage{amssymb}\n\\usepackage[utf8]{inputenc}\n\n\\usepackage[letterpaper,margin=1in]{geometry}\n\\usepackage[OT1]{fontenc}\n\n\\usepackage{amsmath,amssymb,amsthm,amsfonts,latexsym,bbm,xspace,thm-restate}\n\\usepackage{graphicx,float,mathtools,braket,slantsc}\n\\usepackage{enumitem,booktabs,forest,mathdots,soul}\n\\usepackage[useregional]{datetime2}\n\\DTMusemodule{english}{en-US}\n\n\\usepackage{qcircuit,tikz}\n\n\\usepackage[colorlinks,citecolor=blue,,linkcolor=magenta,bookmarks=true]{hyperref}\n\\usepackage[capitalise]{cleveref}\n\n\\usepackage{multicol,array}\n\\usepackage{lipsum,framed}\n\n\\newtheorem{theorem}{Theorem}\n\\newtheorem*{theorem*}{Theorem}\n\\newtheorem{lemma}[theorem]{Lemma}\n\\newtheorem{claim}[theorem]{Claim}\n\\newtheorem{proposition}[theorem]{Proposition}\n\\ne

### Colorcode

In [29]:
RGB = lambda COLOR : ','.join(map(str,COLOR.to_rgb()))

In [30]:
def colorcode(s):
    for keyword in ['in-place', 'In-place', 'In-Place', 'P_f']:
        s = s.replace(keyword, rf" {{\color{{IN_PLACE}} {keyword}}} ")
    for keyword in ['XOR', 'S_f']:
        s = s.replace(keyword, rf" {{\color{{X_O_R}} {keyword}}} ")
    return s

### tex_setup

In [31]:
def tex_setup():
    config.tex_template.add_to_preamble(r"\usepackage{xcolor}")
    config.tex_template.add_to_preamble(rf"\definecolor{{IN_PLACE}}{{rgb}}{{ {RGB(IN_PLACE)} }}")
    config.tex_template.add_to_preamble(rf"\definecolor{{X_O_R}}{{rgb}}{{ {RGB(XOR)} }}")

tex_setup()

### Col Tex Template

In [32]:
DOCUMENT_TEXT_WIDTH = 4.85

def col_template(width=1):
    colTemplate = TexTemplate(
        documentclass=r"\documentclass[preview]{standalone}"
    )
    colTemplate.add_to_preamble(r"\usepackage{xcolor}")
    colTemplate.add_to_preamble(r"\usepackage{ragged2e}")
    colTemplate.add_to_preamble(r"\usepackage{geometry}")
    colTemplate.add_to_preamble(rf"\geometry{{textwidth={DOCUMENT_TEXT_WIDTH*width}in}}")
    colTemplate.add_to_preamble(rf"\definecolor{{IN_PLACE}}{{rgb}}{{ {RGB(IN_PLACE)} }}")
    colTemplate.add_to_preamble(rf"\definecolor{{X_O_R}}{{rgb}}{{ {RGB(XOR)} }}")
    return colTemplate

### Definition

In [33]:
class Definition(VGroup):
    def __init__(
        self,
        header,
        tex,
        box_color=None,
        **kwargs
    ):
        header = Tex(colorcode(header))
        tex = MathTex(colorcode(tex))
        content = VGroup(header, tex).arrange(DOWN)
        super().__init__(
            SurroundingRectangle(content, color=box_color, buff=0.3), 
            content,
            **kwargs
        )

### Bullets

In [34]:
BULLET_SEP = 1
BULLET_TEX_SCALE = 0.7

class Bullets(VGroup):
    
    def __init__(
        self,
        *tex_list, 
        slide=None, 
        width=1, 
        font_scale=1, 
        align_ref=None,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.colTemplate = col_template(width/font_scale)
        self.tex_scale = BULLET_TEX_SCALE*font_scale

        self.align_ref = align_ref

        self._prior_newlines = BULLET_SEP*1.5

        for tex in tex_list:
            self.add_line(tex)

        if slide is not None:
            self.play(slide)
            

    def add_line(self, tex):
        if isinstance(tex, int):
            self._prior_newlines = BULLET_SEP * (1 + tex)
            return

        tabs = 0
        while tex[0] == '_':
            tabs += 1
            tex = tex[1:]
        bulletMe = False
        if tex[0] == '.':
            bulletMe = True
            tex = tex[1:]

        tex = colorcode(tex)
        tex = rf"\RaggedRight{{ {tex} }}"
        tex_obj = Tex(
            tex, 
            tex_template=self.colTemplate,
        ).scale(self.tex_scale)

        bullet = MathTex("\\bullet").scale(self.tex_scale)
        if bulletMe:
            bullet.next_to(tex_obj, LEFT).align_to(tex_obj, UP)
            bullet.shift(DOWN*0.5*bullet.height)
            tex_obj = VGroup(bullet, tex_obj)

        if self.align_ref is not None:
            prev = self[-1] if len(self) > 0 else self.align_ref
            tex_obj.next_to(prev, self._prior_newlines * DOWN)
            tex_obj.align_to(self.align_ref, LEFT)
        else:
            self.align_ref = tex_obj
        
        tex_obj.shift(RIGHT*tabs*(DEFAULT_MOBJECT_TO_MOBJECT_BUFFER + bullet.width))
            
        self._prior_newlines = BULLET_SEP

        self.add(tex_obj)

    def play(self, slide, animation=FadeIn):
        for line in self:
            slide.play(animation(line))
            slide.next_slide()

### Steps

In [35]:
QUICK_RENDER_STEPS = False

class Steps(VGroup):

    def __init__(
        self,
        scale_factor=BULLET_TEX_SCALE,
        **kwargs,
    ) -> None:
        super().__init__(**kwargs)
        self.scale_factor = scale_factor
        

    def index(arr, key):
        if isinstance(key, int):
            return arr[key]
        for i in key:
            arr = arr[i]
        return arr
    
    
    def prev_cause(self):
        return None if len(self[-1]) == 1 else self[-1][0]
    
    
    def prev_result(self):
        return self[-1][-1]
    

    def add_step(
        self,
        *results,
        cause=None,
        overlap_prev=False,
        linear=False,
        **kwargs,
    ):
        results = map(colorcode, results)
        if cause is not None:
            cause = colorcode(cause)
        
        result_mobj = MathTex(*results, **kwargs).scale(self.scale_factor)
        if len(self) > 0:
            if overlap_prev:
                result_mobj.align_to(self.prev_result(), UL)
            else:
                result_mobj.next_to(self.prev_result(), DOWN).align_to(self.prev_result(), LEFT)
            if cause is None:
                cause = r"\xrightarrow{ \ \ \ }"
            elif cause is "":
                cause = None
            elif cause != '=':
                cause = f"\\xrightarrow{{ {cause} }}"

        self.add(VGroup())
        if cause is not None:
            cause_mobj = MathTex(cause).scale(self.scale_factor).next_to(result_mobj, LEFT)
            cause_mobj.shift(UP*cause_mobj.height*0.4)
            self[-1].add(cause_mobj)
        self[-1].add(result_mobj)

        if linear and len(self) > 1:
            self[-1].next_to(self[-2], RIGHT)
        
        return self

    def play(self, slide, indices=[-1], animation=FadeIn):
        if QUICK_RENDER_STEPS and animation == FadeIn:
            slide.add(*[self.index(i) for i in indices])
            return
        slide.play(*[animation(self.index(i)) for i in indices])
        slide.next_slide()

    def play_diff(self, slide, diff=[]):
        assert(len(self) >= 2)
        pre = self[-2][-1].copy()
        post = self.prev_result()
        post_cause = self.prev_cause()
        slide.play(
            pre.animate.align_to(post, UP)
        )
        
        growing = len(pre) < len(post)
        common = list(range(max(len(pre), len(post))))
        for i in diff:
            common.remove(i)
        common = enumerate(common)
        if not growing:
            common = [(j,i) for (i,j) in common]
        
        if len(pre) == len(post):
            diff = []
        departing, arriving = ([],diff) if growing else (diff,[])

        slide.play(
            FadeIn(post_cause),
            *[FadeOut(pre[i]) for i in departing],
            *[Transform(pre[i], post[j]) for i,j in common],
        )
        if arriving:
            slide.play(
                *[FadeIn(post[i]) for i in arriving],
            )
        
        slide.add(self[-1])
        slide.remove(pre)
        slide.next_slide()

<>:50: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
/tmp/ipykernel_10223/1902256059.py:50: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  elif cause is "":


### StandardArrow

In [36]:
class StandardArrow(Arrow):

    def __init__(
        self,
        *args,
        stroke_width = 2,
        **kwargs,
    ) -> None:
        self.initial_tip_length = DEFAULT_ARROW_TIP_LENGTH*stroke_width/6
        super().__init__(
            *args, 
            stroke_width=stroke_width, 
            tip_length=self.initial_tip_length, 
            **kwargs
        )

    def get_default_tip_length(self) -> float:
        """Returns the default tip_length of the arrow.

        Examples
        --------

        ::

            >>> Arrow().get_default_tip_length()
            0.35
        """
        return self.initial_tip_length

    def _set_stroke_width_from_length(self):
        """Fixes stroke width. I think this can be replaced with a no-op."""
        if config.renderer == RendererType.OPENGL:
            # Mypy does not recognize that the self object in this case
            # is a OpenGLVMobject and that the set_stroke method is
            # defined here:
            # mobject/opengl/opengl_vectorized_mobject.py#L248
            self.set_stroke(  # type: ignore[call-arg]
                width=self.initial_stroke_width,
                recurse=False,
            )
        else:
            self.set_stroke(
                width=self.initial_stroke_width,
                family=False,
            )
        return self

### ShowTexIndices

In [37]:
def ShowTexIndices(self, tex):
    # Observe first level labels
    tex_ = tex.copy().next_to(tex, DOWN)
    self.add(index_labels(tex_))

    # Observe second level labels
    tex__ = tex_.copy().next_to(tex_, DOWN)
    for part in tex__:
        self.add(index_labels(part))

### FlipTransform

In [38]:
def FlipTransform(self, mobject_from=None, mobject_to=None):
    x0 = mobject_from.get_center()[0]
    self.play(Homotopy(lambda x,y,z,t : (np.cos(t*np.pi/2)*(x-x0) + x0, y + 0.1*(x-x0)*np.sin(t*np.pi/2), z), mobject_from, rate_func=rush_into, run_time=0.8))
    self.remove(mobject_from)
    self.play(Homotopy(lambda x,y,z,t : (np.sin(t*np.pi/2)*(x-x0) + x0, y - 0.1*(x-x0)*np.cos(t*np.pi/2), z), mobject_to, rate_func=rush_from, run_time=0.8))

In [39]:
# def _flip(x,y,z,t):
#     return (np.cos(t*np.pi)*x, 0.5*x*np.sin(t*np.pi)+y, z)

# def Flip(mobject=None, *vargs, **kwargs):
#     return Homotopy(_flip, mobject, *vargs, **kwargs)

### GroverQuery

In [40]:
def GroverQuery(self, bar_chart):
    def _negate(x,y,z,t):
        x,y = bar_chart.point_to_coords([x,y,z])
        return bar_chart.coords_to_point(x - 0.3*y*np.sin(t*np.pi), np.cos(t*np.pi)*y)
    idx = 0 # index of bar of marked element
    bar = bar_chart.bars[idx]
    self.play(Homotopy(_negate, bar, rate_func=rush_into, run_time=0.8))
    bar_chart.bars.remove(bar)
    bar_chart.bars.insert(idx, bar_chart._create_bar(idx, -bar_chart.values[idx]))
    bar_chart._update_colors()
    bar_chart.values[0] *= -1

### GroverDiffuseTransform

In [41]:
def GroverDiffuseTransform(self, bar_chart):
    avg = np.average(bar_chart.values)
    new_values = 2*avg - np.array(bar_chart.values)
    def _diffuse_from(x,y,z,t):
        x,y = bar_chart.point_to_coords([x,y,z])
        return bar_chart.coords_to_point(x - 0.3*(y-avg)*np.sin(t*np.pi/2), np.cos(t*np.pi/2)*(y-avg) + avg)
    def _diffuse_to(x,y,z,t):
        x,y = bar_chart.point_to_coords([x,y,z])
        return bar_chart.coords_to_point(x + 0.3*(y-avg)*np.cos(t*np.pi/2), np.sin(t*np.pi/2)*(y-avg) + avg)

    # Average
    avg_line = bar_chart.plot(lambda x : np.average(bar_chart.values), stroke_width=1)
    self.play(Create(avg_line))
    
    # Diffuse
    self.play(*[Homotopy(_diffuse_from, bar, rate_func=rush_into, run_time=0.8) for bar in bar_chart.bars])
    for i,bar in enumerate(bar_chart.bars):
        bar_chart.bars.remove(bar)
        bar_chart.bars.insert(i, bar_chart._create_bar(i, new_values[i]))
    bar_chart._update_colors()
    bar_chart.values[:len(new_values)] = new_values
    self.play(*[Homotopy(_diffuse_to, bar, rate_func=rush_from, run_time=0.8) for bar in bar_chart.bars])

    # Cleanup
    self.play(FadeOut(avg_line))

### ShiftBars

In [42]:
def ShiftBars(self, bar_chart):
    self.play(
        *[
            bar.animate.next_to(
            bar_chart.c2p(i+1 + 0.5, 0), 
            (UP if (bar_chart.values[i] >= 0) else DOWN), 
            buff=0) for i,bar in enumerate(bar_chart.bars)
        ]
    )
    bar_colors = [bar.get_color() for bar in bar_chart.bars]
    
    temp_bar = bar_chart.bars[-1].copy().next_to(bar_chart.c2p(0.5, 0), (UP if (bar_chart.values[-1] >= 0) else DOWN), buff=0)
    self.play(Transform(bar_chart.bars[-1], temp_bar, path_arc=90*DEGREES))
    last_val = bar_chart.values[-1]
    # last_color = bar_chart.bar_colors[-1]
    for i in range(len(bar_chart.bars)-1,0,-1):
        bar_chart.bars.remove(bar_chart.bars[i])
        bar_chart.bars.insert(i, bar_chart._create_bar(i, bar_chart.values[i-1]))
        bar_chart.values[i] = bar_chart.values[i-1]
    bar_chart.bars.remove(bar_chart.bars[0])
    bar_chart.bars.insert(0, bar_chart._create_bar(0, last_val))
    bar_chart.values[0] = last_val
    
    bar_chart.bar_colors = [bar_colors[-1]] + bar_colors[0:-1]
    bar_chart._update_colors()
    

### Circuit Braces

In [43]:
def CircuitBraces(ckt, left, right):
    bL = Brace(ckt, LEFT)
    bLt = bL.get_tex(left + r" \rightarrow")
    bR = Brace(ckt, RIGHT)
    bRt = bR.get_tex(r"\rightarrow " + right)
    return VGroup(bL, bLt, bR, bRt)

### OracleCircuit

In [44]:
def OracleCircuit(oracle_tex, left, right, length_tex=r"o(\sqrt{N})", flipped=False):
    flipped = -1 if flipped else 1
    oracle_color, ckt_color = ((IN_PLACE, XOR) if oracle_tex[0] == 'P' else (XOR, IN_PLACE))

    oracle_mathtex = MathTex(oracle_tex)
    oracle_box = SurroundingRectangle(oracle_mathtex, buff=0.2, color=WHITE, fill_opacity=1, fill_color=oracle_color)
    oracle_drawing = VGroup(oracle_box, oracle_mathtex)

    ckt_outline = RoundedRectangle(corner_radius=0.25, height=4.0, width=6.5, fill_opacity=1, fill_color=ckt_color)

    ckt_input = MathTex(left + r" \rightarrow").next_to(ckt_outline, LEFT)
    ckt_output = MathTex(r"\rightarrow " + right).next_to(ckt_outline, RIGHT)
    ckt_input_and_output = VGroup(ckt_input, ckt_output)

    len_line = NumberLine(length=ckt_outline.width, x_range=[0, 1, 1]).next_to(ckt_outline, UP)
    len_label = MathTex(length_tex).next_to(len_line, UP)
    ckt_length = VGroup(len_line, len_label)
    # DoubleArrow(pt1, pt2, tip_shape_end=LineArrowTip, tip_shape_start=LineArrowTip, tip_length=0.1).next_to(ckt_outline, UP)
    
    oracles = VGroup()
    oracles.add(oracle_drawing.move_to(ckt_outline.get_center()).shift(flipped*2.3*LEFT + UP))
    oracles.add(oracles[0].copy().shift(flipped*1.5*RIGHT + DOWN))
    oracles.add(oracles[0].copy().shift(flipped*3*RIGHT))
    oracles.add(oracles[0].copy().shift(flipped*4.5*RIGHT + 2*DOWN))

    ckt = VGroup(ckt_outline, ckt_input_and_output, ckt_length, oracles)

    return ckt

### Get grov and HRY Amps

In [45]:
def get_Grover_amps(t, N=10):
    a_t = np.sin((2*t+1)*np.arctan(1/np.sqrt(N-1)))
    b_t = np.cos((2*t+1)*np.arctan(1/np.sqrt(N-1)))/np.sqrt(N-1)
    return a_t, b_t

In [46]:
def get_HRY_amps(t, N=10):
    a_t = np.sin((t+1)*np.arctan(1/np.sqrt(N-1)))
    b_t = np.cos((t+1)*np.arctan(1/np.sqrt(N-1)))/np.sqrt(N-1)
    return a_t, b_t

## Tests

### Transform Sanity Check

In [47]:
%%manim -ql SanityCheck

class SanityCheck(Scene):
    def construct(self):
        s0 = MathTex(r"a")
        s1 = MathTex(r"\frac{a}{b}")
        s2 = MathTex(r"\frac{b}{a}")
        tms0 = TransformMatchingShapes(s0,s1, key_map={-6351953992051140758:8757917165807693890})
        tms = TransformMatchingShapes(s1,s2)#, key_map={-6351953992051140758:8757917165807693890})
        # s1a = tms.get_mobject_key(tms.get_mobject_parts(s1)[0])
        # s2l = tms.get_mobject_key(tms.get_mobject_parts(s2)[1])
        # tms.key_map = {s1a:s2l}
        print(tms0.get_mobject_parts(s0))
        print(tms0.get_mobject_parts(s1))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s0)[0]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[0]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[1]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[2]))

        print(tms.get_mobject_parts(s1))
        print(tms.get_mobject_parts(s2))

        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[0]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[1]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[2]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[0]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[1]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[2]))
        self.play(tms0)
        self.play(tms)

Manim Community v0.19.0

[VMobjectFromSVGPath]
[VMobjectFromSVGPath, Rectangle, VMobjectFromSVGPath]
-1343967714998759932
-1343967714998759932
-1795526608909711445
1538964478141197471
[VMobjectFromSVGPath, Rectangle, VMobjectFromSVGPath]
[VMobjectFromSVGPath, Rectangle, VMobjectFromSVGPath]
-1343967714998759932
-1795526608909711445
1538964478141197471
1538964478141197471
-1795526608909711445
-1343967714998759932


[09/09/25 16:41:09] INFO     Animation 0 : Using cached data (hash :                           ]8;id=533715;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=18898;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             931832650_1809483027_2358810818)                                                      

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=60429;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=445710;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/SanityCheck/1905288516_2361072207_                         
                             2358810818.mp4'                                                                       

                    INFO     Combining to Movie file.                                      ]8;id=984572;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=366092;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=852678;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=752404;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/SanityCheck.mp4'                                                               
                                                                                                                   

                    INFO     Rendered SanityCheck                                                      ]8;id=649334;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=841784;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   

### Text Test

In [48]:
%%manim -ql TextTest

class TextTest(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        title = Title(colorcode("Search with In-Place Queries"), tex_template=col_template())
        self.add(title)
        self.next_slide()

        # paragraph = Bullets(
        #     r"This is a long string of in-place text meant to span $P_f$ the entire width ${\color{IN_PLACE} x^2}$ of the XOR slide from one margin to the other, at least, that's the hope. Please note if that is not what is observed!", 
        #     align_ref=title
        # )

        # paragraph.play(self)

        bullets = Bullets(
            # "Hello, my name is bob",
            # "Hello, my name is bob",
            # 1,
            # "This sentence is meant to test that text wrapping works properly.",
            ". this is a bullet",
            "_this is an indented bullet \\\\ across multiple lines!! \\\\ So many lines...",
            "_.this is a nested bullet",
            "__.and another",
            "_.and another",
            "$\\phi + 2000$",
            slide=self, width=0.3, font_scale=0.5, align_ref=title
        )

Manim Community v0.19.0

[09/09/25 16:41:11] INFO     Animation 0 : Partial movie file written in                   ]8;id=67337;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=987838;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/TextTest/931832650_2154729144_8197                         
                             77298.mp4'                                                                            

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=552102;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=193071;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/TextTest/1905288516_1443166991_124                         
                             8976031.mp4'                                                                          

[09/09/25 16:41:12] INFO     Animation 2 : Partial movie file written in                   ]8;id=6550;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=761991;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/TextTest/1905288516_1451076942_195                         
                             2314401.mp4'                                                                          

                    INFO     Animation 3 : Partial movie file written in                   ]8;id=600552;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=46153;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/TextTest/1905288516_1443166991_424                         
                             8194740.mp4'                                                                          

...

[09/09/25 16:41:18] INFO     Rendered TextTest                                                         ]8;id=53861;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=777897;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 12 animations                                                                  

[09/09/25 16:41:19] INFO     Generated 6 slides to                                                      ]8;id=88357;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=758978;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TextTest'                            

[09/09/25 16:41:19] INFO     Generated 6 slides to                                                      ]8;id=518492;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=579974;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TextTest'                            

                    INFO     Slide 'TextTest' configuration written in                                  ]8;id=335371;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=44048;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TextTest.json'                             

                    INFO     Slide 'TextTest' configuration written in                                  ]8;id=882684;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=982148;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TextTest.json'                             

### Shift Test

In [49]:
%%manim -ql ShiftTest

class ShiftTest(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        N = 10
        a_t, b_t = get_HRY_amps(1)

        chart = BarChart(
            [a_t] + [b_t]*(N-1),
            y_range=[0, 1, 1],
            bar_colors=[IN_PLACE] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1),
            x_axis_config={"font_size": 30},
            y_length=1,
            x_length=4,
        )

        self.play(FadeIn(chart))
        self.next_slide()
        ShiftBars(self, chart)
        self.next_slide()

Manim Community v0.19.0

[09/09/25 16:41:20] INFO     Animation 0 : Partial movie file written in                   ]8;id=423946;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=411077;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/ShiftTest/931832650_2346884269_223                         
                             132457.mp4'                                                                           

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=449026;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=39999;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/ShiftTest/1905288516_1443166991_22                         
                             17276780.mp4'                                                                         

                    INFO     Animation 2 : Partial movie file written in                   ]8;id=305315;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=503021;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/ShiftTest/1905288516_1051639060_18                         
                             26083455.mp4'                                                                         

[09/09/25 16:41:21] INFO     Animation 3 : Partial movie file written in                   ]8;id=4709;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=801956;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/ShiftTest/1905288516_1549452271_20                         
                             35284394.mp4'                                                                         

...

                    INFO     Rendered ShiftTest                                                        ]8;id=669897;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=784719;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 5 animations                                                                   

[09/09/25 16:41:21] INFO     Generated 2 slides to                                                      ]8;id=697119;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=147150;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ShiftTest'                           

                    INFO     Generated 2 slides to                                                      ]8;id=841114;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=380355;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ShiftTest'                           

                    INFO     Slide 'ShiftTest' configuration written in                                 ]8;id=417247;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=130143;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ShiftTest.json'                            

                    INFO     Slide 'ShiftTest' configuration written in                                 ]8;id=391247;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=475629;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ShiftTest.json'                            

## Overview

### Query Complexity (motivation)
- What is a problem? 
  - Input: some f, Output: some property of f. 
  - Egs: 
    - (1) Given: N-bit integer. Output: is it nonzero? (show arrow to "integer", saying f : [N] -> {0,1}, transform) 
      (show after (2)) Given: function f : [N] -> {0,1}. Output: is there an x such that f(x) = 1?
      mention this is "searching for a 1"
    - (2) Given: a shuffled deck of cards labeled {1,...,N}. Output: location of the "1" card.
      Given permutation f: [N] -> [N]. Output: x such that f(x) = 1.
- "In Complexity theory we often care about how much time the best algorithm to solve a problem would take as the size of the input (N) increases"
- "That's really hard to lower bound. The best technique we have is query complexity."
- In query cxty, we imagine the input is behind a paywall, and we need to pay an oracle every time we want to access bits of it. (oracle pic)
  - I will tell the oracle x and the oracle will tell me f(x), each request is called a query
  <!-- the label of the xth card in the deck. (exchange) -->
  - It's fairly intuitive that, for these specific problems above, N queries are necessary classically, (implying an Omega(N) lower bound on time?).
  - How do we do quantum query complexity? 
    - Our first thought might be |x> -> |f(x)>. 
      - Violates QM if f is not reversible. 
    - Solution: |x>|a> -> |x>|a + f(x)>. -self inverse -equiv to phase oracles
    - Okay, but what if f is a permutation? 
    - Then |x> -> |f(x)> is fine, called in-place queries.

### Key Question of this talk
- Given a permutation f, how do XOR queries and in-place queries to f compare?
  
  <!-- however, the ability to make multiple queries in superposition lets us find "1" with O(sqrt(N)) queries, through what's called Grover's algorithm
  - Understanding this alg is crucial to understanding our alg, so let's briefly review -->

### Table of comparisons
- If we have access to both f and f^{-1}, then both models can simulate the other
- Omega(sqrt(N)) XOR queries required to exactly simulate an in-place query
  - In this paper: Omega(sqrt(N)) in-place to XOR
- IndexErasure (state conversion) O(sqrt(N)) vs O(1)
  - states of the form |x>|f(x)> -> |0>|f(x)>
- SetComp (promise decision problem) O(N^{1/7}) vs O(1)
  - venn diagram highlighting: two large sets are the same or have large symmetric difference
- Unstructured search (search problem) O(sqrt(N)) vs conjectured Omega(N). (cross out conj, write ours) In this paper: O(sqrt(N)).
  - N shuffled cards, locate "1"
- Trashy Simon's (promise search problem) O(log(N)) vs conjectured Omega(sqrt(N)) ### TODO: verify sqrt(N) correct!

### Grover's
- Our problem: Given a shuffled deck of cards labeled [N], find 1. 
- (Show 12345 example truth table with f, circle inv of 1) 
- (transform problem) given a permutation f : [N] -> [N], find x^* := f^{-1}(1).
- (disappear truth table)
- In Grover's algorithm, we start off with a uniform superposition over all x in [N] (bar chart)
- Repeatedly alternate between the following two operations:
  - Query, which negates the amplitude on x^* (flip)
  - Grover's Diffusion operator, which flips all amplitudes about their average (draw line, flip)
  - (quickly evolve a few times until x^* peaks) after ~sqrt N iterations of this, we will have made ~sqrt N queries and the probability of seeing x^* when we measure.

### Where Grover's fails
- key step is being able to take |x> -> (-1)^{f(x) = 1} |x>
- |x>|0> -> |x>|f(x)> -> (-1)^{f(x) = 1}|x>|f(x)> (->) |x>|0>  <- (new slide) function erasure is hard
- Without being able to erase f, interference fails

### In fact, we show func erasure is hard!
- pf omitted here.

### Our Alg
- Do the math, have a box reminding people what in-place and XOR do
- |x> -> |x>|x>|0> -> |x>|f(x)>|0> -> |x*>|1>|1> + sum|x>|f(x)>|0>
- -> |x*>|1>|1> + sum|f(x)>|f(x)>|0> -> |x*>|1>|1> + sum|f(x)>|0>|0>  -> |x*>|1>|1> + sum|f(x)>|1>|0> -> |x*>|1> + sum|f(x)>|0>
- convert to bars

### Step Thru Our Alg
- show bars

### Kashefi
- 
- Key fact: perm inv LBs for both Sf [Ambainis] and Pf [FK18]

### Kashefi Pfs (3 slides)
- Inverting circuits
- Kashefi arg (|x>|a> -[Sf]- |x>|f(x)> in corner so we can flip it to drive home that Sf inv = Sf)
- Our arg (|x> -[Pf]- |f(x)> in corner so we can flip it to drive home that Pf inv = P{f_inv})
- => function erasure is hard (analog of index erasure)

### Sf Pf box
- Grovers + Kashefi => Theta(sqrt(N)) XOR to in-place
- Our Alg + our arg => Theta(sqrt(N)) in-place to XOR

### Summary
- Unstructured Search takes Theta(sqrt(N)) queries (in-place or XOR) (algorithm)
- Function Erasure takes O(1) XOR Queries and O(sqrt(N)) in-place queries (kashefi)
- More generally, Sf Pf Sfinv Pfinv all req sqrt(N) queries to simulate each other
- Future work: decision problem separations
  - candidates: simon, emb perm inv


## TitleSlide

In [50]:
%%manim -ql TitleSlide

class TitleSlide(Slide):
    def construct(self):
        # self.wait_time_between_slides = 1
        title = Tex(r"\textsc{Quantum Search with In-Place Queries}").scale(1.2).shift(2*UP)
        line = Underline(title)
        subtitle = Tex(r"Blake Holman, \textbf{Ronak Ramachandran}, and Justin Yirka").scale(0.75)
        subtitle.next_to(title, DOWN, buff=0.5)
        ronak = ImageMobject("ronak.png").scale_to_fit_height(2.5).next_to(subtitle, DOWN, buff=0.5)
        blake = ImageMobject("blake.jpg").scale_to_fit_height(2.5).next_to(ronak, LEFT, buff=1)
        justin = ImageMobject("jirka.jpg").scale_to_fit_height(2.5).next_to(ronak, RIGHT, buff=1)
        remail = Tex(r"ronakr@utexas.edu").next_to(ronak, DOWN).scale(0.5)
        bemail = Tex(r"holman14@purdue.edu").next_to(blake, DOWN).scale(0.5)
        jemail = Tex(r"justin@blanqet.net").next_to(justin, DOWN).scale(0.5)
        # self.play(Add(title), Write(subtitle), Succession(FadeIn(blake), FadeIn(ronak), FadeIn(justin), run_time=1.8))
        self.add(
            title, line, subtitle,
            blake, ronak, justin,
            bemail, remail, jemail
        )
        self.wait()
        self.next_slide()

Manim Community v0.19.0

[09/09/25 16:41:23] INFO     Animation 0 : Using cached data (hash :                           ]8;id=672585;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=202150;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             931832650_722531385_1866445582)                                                       

[09/09/25 16:41:24] INFO     Combining to Movie file.                                      ]8;id=86377;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=954251;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=900424;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=594891;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/TitleSlide.mp4'                                                                
                                                                                                                   

                    INFO     Rendered TitleSlide                                                       ]8;id=138067;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=816527;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 1 animations                                                                   

[09/09/25 16:41:24] INFO     Generated 1 slides to                                                      ]8;id=845050;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=388746;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TitleSlide'                          

                    INFO     Generated 1 slides to                                                      ]8;id=894812;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=489245;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TitleSlide'                          

                    INFO     Slide 'TitleSlide' configuration written in                                ]8;id=609509;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=223529;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TitleSlide.json'                           

                    INFO     Slide 'TitleSlide' configuration written in                                ]8;id=863386;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=774689;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TitleSlide.json'                           

# Introduction

## Complexity

In [51]:
%%manim -ql Complexity

class Complexity(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Introduction"))
        self.add(title)
        self.wait()
        self.next_slide()

        bullets = Bullets(
            r"Life is full of problems",
            r".Eg. Find the last available parking spot",
            r"Problems take the form: Given some input $f$, compute some property of $f$",
            r".Eg. \textsc{UnstructuredSearch}: Given a function $f$, find $f^{-1}(1)$",
            r"Complexity theorists want to know how much time it takes to solve different problems",
            r".Hard to pin down",
            r"One proxy for time is \textbf{query complexity}",
            slide=self, align_ref=title
        )

Manim Community v0.19.0

                    INFO     Animation 0 : Partial movie file written in                   ]8;id=475355;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=129475;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Complexity/931832650_722531385_990                         
                             003833.mp4'                                                                           

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=848108;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=798158;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Complexity/1905288516_1443166991_6                         
                             61816014.mp4'                                                                         

[09/09/25 16:41:26] INFO     Animation 2 : Partial movie file written in                   ]8;id=856265;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=288516;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Complexity/1905288516_1130047418_6                         
                             87751752.mp4'                                                                         

[09/09/25 16:41:27] INFO     Animation 3 : Partial movie file written in                   ]8;id=797667;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=868857;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Complexity/1905288516_1443166991_2                         
                             553863330.mp4'                                                                        

...

                    INFO     Rendered Complexity                                                       ]8;id=322011;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=844382;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 16 animations                                                                  

[09/09/25 16:41:43] INFO     Generated 8 slides to                                                      ]8;id=973939;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=905060;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Complexity'                          

[09/09/25 16:41:43] INFO     Generated 8 slides to                                                      ]8;id=252173;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=713713;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Complexity'                          

[09/09/25 16:41:44] INFO     Slide 'Complexity' configuration written in                                ]8;id=401695;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=226227;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Complexity.json'                           

[09/09/25 16:41:44] INFO     Slide 'Complexity' configuration written in                                ]8;id=540363;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=101864;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Complexity.json'                           

## QueryComplexity

In [52]:
%%manim -ql QueryComplexity

class QueryComplexity(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Query Complexity"))
        self.add(title)
        self.wait()
        self.next_slide()

        bullets = Bullets(
            "Imagine the input $f$ is stuck behind a paywall",
            ".Can ask questions (queries) about the input of the form ``$x$''",
            ".Receive answers of the form ``$f(x)$''",
            ".Each question is expensive",
            ".Everything else is free",
            1,
            r"The \textbf{query complexity} of a problem is the minimum number of queries necessary to solve the problem",
            slide=self, align_ref=title
        )

Manim Community v0.19.0

                    INFO     Animation 0 : Partial movie file written in                   ]8;id=812699;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=896813;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/QueryComplexity/931832650_72253138                         
                             5_1494000380.mp4'                                                                     

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=62674;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=145782;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/QueryComplexity/1905288516_1443166                         
                             991_2609267879.mp4'                                                                   

[09/09/25 16:41:47] INFO     Animation 2 : Partial movie file written in                   ]8;id=679695;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=623624;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/QueryComplexity/1905288516_1541848                         
                             93_4182071359.mp4'                                                                    

                    INFO     Animation 3 : Partial movie file written in                   ]8;id=561224;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=397700;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/QueryComplexity/1905288516_1443166                         
                             991_3892269195.mp4'                                                                   

...

                    INFO     Rendered QueryComplexity                                                  ]8;id=88379;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=403033;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 14 animations                                                                  

[09/09/25 16:42:01] INFO     Generated 7 slides to                                                      ]8;id=929443;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=70633;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/QueryComplexity'                     

[09/09/25 16:42:01] INFO     Generated 7 slides to                                                      ]8;id=203501;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=16965;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/QueryComplexity'                     

                    INFO     Slide 'QueryComplexity' configuration written in                           ]8;id=998685;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=492870;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/QueryComplexity.json'                      

                    INFO     Slide 'QueryComplexity' configuration written in                           ]8;id=760705;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=802415;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/QueryComplexity.json'                      

## QueryModels

In [53]:
%%manim -ql QueryModels

class QueryModels(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Quantum Query Models"))
        self.add(title)
        self.wait()
        self.next_slide()

        sep = 0.5

        bullets = Bullets(
            "To perform a query, we need to be able to query $x$ and receive back $f(x)$",
            "Quantum mechanics is reversible, so quantum queries need to be reversible",
            r".If $f$ is not reversible, the map $|x\rangle \rightarrow |f(x)\rangle$ is not unitary",
            r"_Solution: ",
            slide=self, align_ref=title
        )

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        ).scale(0.7).next_to(bullets, DOWN).set_x(0)
        self.play(FadeIn(xor_defn))
        self.next_slide()

        bullets2 = Bullets(
            r"When $f$ is a permutation from $[N]$ to $[N]$, then $|x\rangle \rightarrow |f(x)\rangle$ is unitary",
        ).next_to(xor_defn, DOWN).align_to(title, LEFT)
        bullets2.play(self)

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        ).scale(0.7).next_to(bullets2, DOWN).set_x(0)
        self.play(FadeIn(in_place_defn))
        self.next_slide()
        

Manim Community v0.19.0

                    INFO     Animation 0 : Partial movie file written in                   ]8;id=734004;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=717076;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/QueryModels/931832650_722531385_25                         
                             65132333.mp4'                                                                         

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=740156;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=814211;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/QueryModels/1905288516_1443166991_                         
                             3535811445.mp4'                                                                       

[09/09/25 16:42:04] INFO     Animation 2 : Partial movie file written in                   ]8;id=548440;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=303529;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/QueryModels/1905288516_3990117430_                         
                             548100106.mp4'                                                                        

                    INFO     Animation 3 : Partial movie file written in                   ]8;id=397672;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=584014;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/QueryModels/1905288516_1443166991_                         
                             1369093822.mp4'                                                                       

...

                    INFO     Rendered QueryModels                                                      ]8;id=798793;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=717202;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 16 animations                                                                  

[09/09/25 16:42:24] INFO     Generated 8 slides to                                                      ]8;id=271694;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=478620;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/QueryModels'                         

[09/09/25 16:42:24] INFO     Generated 8 slides to                                                      ]8;id=927354;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=971288;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/QueryModels'                         

                    INFO     Slide 'QueryModels' configuration written in                               ]8;id=30668;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=861832;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/QueryModels.json'                          

                    INFO     Slide 'QueryModels' configuration written in                               ]8;id=588367;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=103582;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/QueryModels.json'                          

## KeyQuestion

In [54]:
%%manim -ql KeyQuestion

class KeyQuestion(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        question = Tex(
            r"Given a \textbf{permutation} $f$, how do \textbf{XOR queries} \\ and \textbf{in-place queries} to $f$ compare?",
            substrings_to_isolate=[r"\textbf{XOR queries}", r"\textbf{in-place queries}"]
        )
        question.set_color_by_tex(r"XOR", XOR)
        question.set_color_by_tex(r"in-place", IN_PLACE)
        self.play(Write(question))
        self.next_slide()

Manim Community v0.19.0

[09/09/25 16:42:26] INFO     Animation 0 : Using cached data (hash :                           ]8;id=309476;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=462761;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             931832650_2382438420_223132457)                                                       

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=719444;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=38836;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1443166991_4246822100)                                                     

                    INFO     Combining to Movie file.                                      ]8;id=182029;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=830259;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=589989;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=641459;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/KeyQuestion.mp4'                                                               
                                                                                                                   

                    INFO     Rendered KeyQuestion                                                      ]8;id=758382;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=335128;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   

[09/09/25 16:42:26] INFO     Generated 1 slides to                                                      ]8;id=571955;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=248748;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/KeyQuestion'                         

                    INFO     Generated 1 slides to                                                      ]8;id=586865;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=708441;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/KeyQuestion'                         

                    INFO     Slide 'KeyQuestion' configuration written in                               ]8;id=329590;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=813067;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/KeyQuestion.json'                          

                    INFO     Slide 'KeyQuestion' configuration written in                               ]8;id=470083;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=935106;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/KeyQuestion.json'                          

## Comparisons

In [55]:
%%manim -ql Comparisons

class Comparisons(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Comparing XOR and in-place queries"))
        self.add(title)
        self.wait()
        self.next_slide()

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )

        sep = 1.5
        oracle_defns = VGroup(
            xor_defn, in_place_defn
        ).scale(BULLET_TEX_SCALE).arrange(RIGHT*sep).next_to(title, DOWN*sep)
        self.play(FadeIn(oracle_defns))
        self.next_slide()

        bullets = Bullets(
            r"With access to both $f$ and $f^{-1}$, XOR and in-place are equivalent",
            r"With only access to $f$, in-place queries look very powerful [KKVB02]:",
            r".There is an $f$ where",
            r"_.XOR queries to $f$ are easy to implement",
            r"_.In-place queries to $f$ efficiently solve (Non-Automorphic) Graph Isomorphism",
            r"_.Simulating in-place queries with few XOR queries $\implies$ efficient algorithms",
            r".$\Omega(\sqrt{N})$ XOR queries to $f$ are necessary to simulate one in-place query to $f$",
        ).next_to(oracle_defns, DOWN*sep).align_to(title, LEFT).play(self)

Manim Community v0.19.0

[09/09/25 16:42:27] INFO     Animation 0 : Using cached data (hash :                           ]8;id=657172;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=689586;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             931832650_722531385_3357819802)                                                       

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=648495;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=523516;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1443166991_1221851079)                                                     

[09/09/25 16:42:28] INFO     Animation 2 : Using cached data (hash :                           ]8;id=826027;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=163511;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_161108985_1391712155)                                                      

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=609355;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=532649;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1443166991_2768872009)                                                     

...

                    INFO     Rendered Comparisons                                                      ]8;id=89403;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=439727;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 18 animations                                                                  

[09/09/25 16:42:56] INFO     Generated 9 slides to                                                      ]8;id=984365;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=559577;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Comparisons'                         

[09/09/25 16:42:56] INFO     Generated 9 slides to                                                      ]8;id=441880;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=579436;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Comparisons'                         

[09/09/25 16:42:57] INFO     Slide 'Comparisons' configuration written in                               ]8;id=244427;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=485031;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Comparisons.json'                          

[09/09/25 16:42:57] INFO     Slide 'Comparisons' configuration written in                               ]8;id=679086;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=35373;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Comparisons.json'                          

## ComparisonTable

In [56]:
%%manim -ql ComparisonTable

class ComparisonTable(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Are in-place queries stronger than XOR queries?"))
        self.add(title)
        self.wait()
        self.next_slide()

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )

        perm_inv_defn = Definition(
            r"\textsc{PermutationInversion}",
            r"\text{Given $f$, find $f^{-1}(1)$}",
        )

        idx_erase_defn = Definition(
            r"\textsc{IndexErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |0\rangle |f(x)\rangle",
        )

        func_erase_defn = Definition(
            r"\textsc{FunctionErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |x\rangle |0\rangle",
        )

        idx_erase_defn[0].match_height(perm_inv_defn[0])
        func_erase_defn[0].match_height(perm_inv_defn[0])

        def ul(upper, ucite, lower, lcite):
            u = Tex(rf"$O({upper})$ [{ucite}]")
            l = Tex(rf"$\Omega({lower})$ [{lcite}]").next_to(u, DOWN).align_to(u, LEFT)
            return VGroup(u, l)

        xp_ul = ul(r"\sqrt{N}", "Gro96", r"\sqrt{N}", "Amb02")
        xi_ul = ul(r"\sqrt{N}", "Gro96", r"\sqrt{N}", "AMRR11")
        xf_ul = MathTex(rf"\Theta(1)").scale(2)
        ip_ul = ul(r"\sqrt{N}", r"H\textbf{R}Y25", r"\sqrt{N}", "FK18][BY23")
        ii_ul = MathTex(rf"\Theta(1)").scale(2)
        if_ul = ul(r"\sqrt{N}", r"H\textbf{R}Y25", r"\sqrt{N}", r"H\textbf{R}Y25")

        ip_ul[0].set_color(RED)
        if_ul.match_color(ip_ul[0])
        func_erase_defn.match_color(ip_ul[0])

        table = VGroup(
            VMobject(), idx_erase_defn, perm_inv_defn, func_erase_defn,
            xor_defn, xi_ul, xp_ul, xf_ul,
            in_place_defn, ii_ul, ip_ul, if_ul,
        )

        table.arrange_in_grid(
            col_alignments="rccc",
            row_alignments="dcc",
        )

        conj = Tex(
            rf"[FK18]: Is $O(N)$ optimal?"
        ).next_to(ip_ul[1], UP).align_to(ip_ul[1], LEFT)

        (table+conj).match_width(title).shift(UP*0.5)
        cross = Cross(conj)

        open_prob = Tex(
            colorcode(r"STILL OPEN: \\ Does any \textbf{decision problem} require more in-place queries than XOR queries?")
        ).scale(BULLET_TEX_SCALE).next_to(table, DOWN*2)
        
        def fadein(*x):
            self.play(*[FadeIn(item) for item in x])
            self.next_slide()

        fadein(xor_defn, in_place_defn)
        fadein(idx_erase_defn)
        fadein(ii_ul)
        fadein(xi_ul[1])
        fadein(xi_ul[0])

        fadein(perm_inv_defn)
        fadein(xp_ul)
        fadein(ip_ul[1])
        fadein(conj)
        fadein(cross)
        self.play(FadeOut(conj), FadeOut(cross), FadeIn(ip_ul[0]))
        self.next_slide()

        fadein(func_erase_defn)
        fadein(xf_ul)
        fadein(if_ul[1])
        fadein(if_ul[0])

        fadein(open_prob)

Manim Community v0.19.0

                    INFO     Animation 0 : Partial movie file written in                   ]8;id=568690;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=71062;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/ComparisonTable/931832650_72253138                         
                             5_2474379789.mp4'                                                                     

[09/09/25 16:42:58] INFO     Animation 1 : Partial movie file written in                   ]8;id=64497;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=202373;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/ComparisonTable/1905288516_1443166                         
                             991_3182230294.mp4'                                                                   

[09/09/25 16:43:01] INFO     Animation 2 : Partial movie file written in                   ]8;id=610317;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=913074;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/ComparisonTable/1905288516_6199988                         
                             79_3373764845.mp4'                                                                    

[09/09/25 16:43:02] INFO     Animation 3 : Partial movie file written in                   ]8;id=74601;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=566325;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/ComparisonTable/1905288516_1443166                         
                             991_1090464775.mp4'                                                                   

...

                    INFO     Rendered ComparisonTable                                                  ]8;id=595496;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=18209;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 34 animations                                                                  

[09/09/25 16:43:47] INFO     Generated 17 slides to                                                     ]8;id=566043;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=547723;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ComparisonTable'                     

[09/09/25 16:43:47] INFO     Generated 17 slides to                                                     ]8;id=606635;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=832852;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ComparisonTable'                     

                    INFO     Slide 'ComparisonTable' configuration written in                           ]8;id=543707;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=56825;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ComparisonTable.json'                      

                    INFO     Slide 'ComparisonTable' configuration written in                           ]8;id=148392;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=701132;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ComparisonTable.json'                      

# Why Grover Fails --> FE LB

## StepThruGroverBar

In [57]:
%%manim -ql StepThruGroverBar

class StepThruGroverBar(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        title = Title("Grover's Algorithm")
        self.add(title)
        self.wait()
        self.next_slide()

        N = 10
        T = int(np.ceil(np.pi/4 * np.sqrt(N)))

        chart = BarChart(
            [1/np.sqrt(N)]*N,
            y_range=[-1, 1, 1],
            bar_colors=[XOR] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1), #, "1", "$f(1)$",r"$f^{2}(1)$", r"$f^{3}(1)$", "", r"$\dots$", "", r"$f^{-2}(x^*)$", r"$f^{-1}(x^*)$"],
            x_axis_config={"font_size": 36},
        )
        y_axis_label = chart.get_y_axis_label(Tex("Amplitude").rotate(90*DEGREES), edge=LEFT, direction=LEFT, buff=0)
        VGroup(chart, y_axis_label).scale(0.7).next_to(title, DOWN*2)

        query_box = Definition(
            r"Query $f$",
            r"\text{Negate $|x^*\rangle$ amplitude}",
            box_color=XOR
        )

        diffuse_box = Definition(
            r"Diffuse",
            r"\text{Flip about average}"
        ).match_height(query_box)

        op_boxes = VGroup(
            query_box, 
            diffuse_box,
        ).arrange(RIGHT).scale(0.7).next_to(chart, DOWN)

        inactive_opacity = 0.2
        inactive_darkness = 1 - inactive_opacity

        query_box[0].set_color(XOR.darker(inactive_darkness))
        query_box[1].set_opacity(inactive_opacity)
        diffuse_box[0].set_color(WHITE.darker(inactive_darkness))
        diffuse_box[1].set_opacity(inactive_opacity)

        self.play(
            DrawBorderThenFill(chart), 
            DrawBorderThenFill(y_axis_label),
            FadeIn(op_boxes),
        )
        self.next_slide()

        a_t,b_t = get_Grover_amps(0,N)
        for t in range(1,T):
            # Query
            query_box[0].set_color(XOR)
            query_box[1].set_opacity(1)
            diffuse_box[0].set_color(WHITE.darker(inactive_darkness))
            diffuse_box[1].set_opacity(inactive_opacity)
            
            GroverQuery(self, chart)
            self.next_slide()

            # Diffuse
            query_box[0].set_color(XOR.darker(inactive_darkness))
            query_box[1].set_opacity(inactive_opacity)
            diffuse_box[0].set_color(WHITE)
            diffuse_box[1].set_opacity(1)

            a_t,b_t = get_Grover_amps(t,N)
            GroverDiffuseTransform(self, chart)
            self.next_slide()

        query_box[0].set_color(XOR.darker(inactive_darkness))
        query_box[1].set_opacity(inactive_opacity)
        diffuse_box[0].set_color(WHITE.darker(inactive_darkness))
        diffuse_box[1].set_opacity(inactive_opacity)

        question = Bullets(r"Where does this go wrong with in-place queries?").next_to(op_boxes, DOWN)

        self.play(FadeIn(question))
        self.next_slide()

        self.play(
            query_box[0].animate.set_color(XOR),
            query_box[1].animate.set_opacity(1),
            Circumscribe(query_box[0], color=IN_PLACE, fade_out=True),
        )
        self.next_slide()


Manim Community v0.19.0

[09/09/25 16:43:48] INFO     Animation 0 : Partial movie file written in                   ]8;id=232633;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=420432;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruGroverBar/931832650_722531                         
                             385_4276835967.mp4'                                                                   

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=38188;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=990060;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruGroverBar/1905288516_14431                         
                             66991_3725775767.mp4'                                                                 

[09/09/25 16:43:51] INFO     Animation 2 : Partial movie file written in                   ]8;id=491912;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=33036;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruGroverBar/1905288516_36198                         
                             58076_1371791780.mp4'                                                                 

[09/09/25 16:43:52] INFO     Animation 3 : Partial movie file written in                   ]8;id=722618;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=771566;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruGroverBar/1905288516_14431                         
                             66991_3622785665.mp4'                                                                 

...

                    INFO     Rendered StepThruGroverBar                                                ]8;id=551143;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=398383;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 22 animations                                                                  

[09/09/25 16:44:13] INFO     Generated 8 slides to                                                      ]8;id=758584;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=179909;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruGroverBar'                   

[09/09/25 16:44:13] INFO     Generated 8 slides to                                                      ]8;id=72708;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=307916;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruGroverBar'                   

                    INFO     Slide 'StepThruGroverBar' configuration written in                         ]8;id=216857;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=15086;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruGroverBar.json'                    

                    INFO     Slide 'StepThruGroverBar' configuration written in                         ]8;id=662075;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=101516;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruGroverBar.json'                    

## GroverBreakdown

In [58]:
%%manim -ql GroverBreakdown

class GroverBreakdown(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Why can't we Grover's search with in-place queries?"))
        self.add(title)
        self.wait()
        self.next_slide()

        query_tex_cases = MathTex(r"|x\rangle \rightarrow \begin{cases} -|x\rangle & \text{if $f(x) = 1$} \\ |x\rangle & \text{otherwise} \end{cases}")
        query_tex = MathTex(r"|x\rangle \rightarrow (-1)^{f(x)=1}|x\rangle")
        self.play(Write(query_tex_cases))
        self.next_slide()
        self.play(ReplacementTransform(query_tex_cases, query_tex))
        self.next_slide()

        self.play(query_tex.animate.shift(2*UP))
        steps = VGroup()
        steps.add(MathTex(
            r"|x\rangle",
            r"|0\rangle",
        ))
        steps.add(MathTex(
            r"\xrightarrow{S_f}",
            r"|x\rangle|f(x)\rangle",
        ).next_to(steps[-1], DOWN).align_to(query_tex, LEFT))
        steps.add(MathTex(
            r"\xrightarrow{\phantom{S_f}} (-1)^{f(x)=1}",
            r"|x\rangle|f(x)\rangle",
        ).next_to(steps[-1], DOWN).align_to(query_tex, LEFT))
        steps.add(MathTex(
            r"\xrightarrow{S_f} (-1)^{f(x)=1}",
            r"|x\rangle",
            r"|0\rangle",
        ).next_to(steps[-1], DOWN).align_to(query_tex, LEFT))
        steps[0].align_to(steps[1][1], LEFT)
        steps[1][0][0:2].set_fill(color=XOR)
        steps[3][0][0:2].set_fill(color=XOR)

        # ShowTexIndices(self, steps)

        self.play(FadeIn(steps[0][0]))
        self.next_slide()
        self.play(FadeIn(steps[0][1]))
        self.next_slide()
        self.play(FadeIn(steps[1]))
        self.next_slide()
        self.play(FadeIn(steps[2]))
        self.next_slide()
        self.play(FadeIn(steps[3][0]), FadeIn(steps[3][1]), FadeIn(steps[3][2]))
        self.next_slide()
        # self.play(FadeOut(steps[3][-1]))
        # self.next_slide()

        self.play(Circumscribe(VGroup(steps[2], steps[3]), color=RED, fade_out=True))

        func_eras = MathTex(
            r"|x\rangle|f(x)\rangle",
            r"\rightarrow",
            r"|x\rangle",
            r"|0\rangle",
        )
        self.play(
            FadeOut(query_tex, steps[0], steps[1], steps[2][0], steps[3][0]),
            ReplacementTransform(steps[2][1], func_eras[0]),
            FadeIn(func_eras[1]),
            ReplacementTransform(steps[3][1], func_eras[2]),
            ReplacementTransform(steps[3][2], func_eras[3])
        )
        self.next_slide()

        func_eras_title = Tex(r"\textsc{FunctionErasure}").next_to(func_eras, 2*UP)
        self.play(Write(func_eras_title))
        self.next_slide()

        lower_bound = Bullets(
            r"We prove \textsc{FunctionErasure} requires $\Omega(\sqrt{N})$ in-place queries!"
        ).next_to(func_eras, DOWN*2)
        self.play(FadeIn(lower_bound))
        self.next_slide()


Manim Community v0.19.0

[09/09/25 16:44:14] INFO     Animation 0 : Partial movie file written in                   ]8;id=71156;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=107455;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/GroverBreakdown/931832650_72253138                         
                             5_4066280429.mp4'                                                                     

[09/09/25 16:44:15] INFO     Animation 1 : Partial movie file written in                   ]8;id=733863;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=432133;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/GroverBreakdown/1905288516_1443166                         
                             991_81983281.mp4'                                                                     

[09/09/25 16:44:16] INFO     Animation 2 : Partial movie file written in                   ]8;id=139230;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=590760;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/GroverBreakdown/1905288516_2712564                         
                             802_916688624.mp4'                                                                    

[09/09/25 16:44:17] INFO     Animation 3 : Partial movie file written in                   ]8;id=61191;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=728212;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/GroverBreakdown/1905288516_1443166                         
                             991_3982130232.mp4'                                                                   

...

                    INFO     Rendered GroverBreakdown                                                  ]8;id=190348;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=3552;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 24 animations                                                                  

[09/09/25 16:44:36] INFO     Generated 11 slides to                                                     ]8;id=735119;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=533048;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/GroverBreakdown'                     

[09/09/25 16:44:36] INFO     Generated 11 slides to                                                     ]8;id=21827;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=905862;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/GroverBreakdown'                     

                    INFO     Slide 'GroverBreakdown' configuration written in                           ]8;id=417151;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=231657;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/GroverBreakdown.json'                      

                    INFO     Slide 'GroverBreakdown' configuration written in                           ]8;id=852886;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=408903;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/GroverBreakdown.json'                      

## FuncErasToXOR

In [59]:
%%manim -ql FuncErasToXOR

class FuncErasToXOR(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Why is \textsc{FunctionErasure} hard with in-place queries?"), font_size=40)
        self.add(title)
        self.wait()
        self.next_slide()

        bullets = Bullets(
            r"Proof sketch:",
            r".Solving \textsc{FunctionErasure} $\implies$ simulating XOR queries",
            slide=self, align_ref=title
        )
        bullets2 = Bullets(
            r".Simulating XOR queries $\implies$ solving \textsc{PermutationInversion}",
            r".Solving \textsc{PermutationInversion} requires $\Omega(\sqrt{N})$ in-place queries [FK18]",
            slide=self, align_ref=bullets
        )
        self.play(
            bullets2.animate.shift(DOWN*3)
        )
        steps = Steps(scale_factor=0.7)
        steps.add_step(
            r"|x\rangle",
            r"|a\rangle",
        ).next_to(bullets, DOWN).play(self)

        self.remove(steps[-1])
        steps.add_step(
            r"|x\rangle",
            r"|a\rangle",
            r"|0\rangle",
            overlap_prev=True, cause=""
        ).play_diff(self, diff=[2])
        steps.remove(steps[-2])

        steps.add_step(
            r"|x\rangle",
            r"|a\rangle",
            r"|x\rangle",
            cause=r"\text{CNOT}"
        ).play_diff(self)

        steps.add_step(
            r"|x\rangle",
            r"|a\rangle",
            r"|f(x)\rangle",
            cause=r"P_f"
        ).play_diff(self)

        steps.add_step(
            r"|x\rangle",
            r"|a \oplus f(x)\rangle",
            r"|f(x)\rangle",
            cause=r"\text{CNOT}"
        ).play_diff(self)

        steps.add_step(
            r"|x\rangle",
            r"|a \oplus f(x)\rangle",
            r"|0\rangle",
            cause=r"\text{FE}"
        ).play_diff(self)
        
        self.play(FadeOut(steps.prev_result()[2]))
        self.next_slide()

Manim Community v0.19.0

[09/09/25 16:44:37] INFO     Animation 0 : Partial movie file written in                   ]8;id=347179;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=294798;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/FuncErasToXOR/931832650_722531385_                         
                             2071718580.mp4'                                                                       

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=485805;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=11364;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/FuncErasToXOR/1905288516_144316699                         
                             1_2212230234.mp4'                                                                     

[09/09/25 16:44:38] INFO     Animation 2 : Partial movie file written in                   ]8;id=523801;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=43637;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/FuncErasToXOR/1905288516_318726981                         
                             3_4251113445.mp4'                                                                     

                    INFO     Animation 3 : Partial movie file written in                   ]8;id=478717;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=200799;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/FuncErasToXOR/1905288516_144316699                         
                             1_69923168.mp4'                                                                       

...

                    INFO     Rendered FuncErasToXOR                                                    ]8;id=847186;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=52016;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 31 animations                                                                  

[09/09/25 16:46:11] INFO     Generated 12 slides to                                                     ]8;id=799840;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=272924;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/FuncErasToXOR'                       

[09/09/25 16:46:11] INFO     Generated 12 slides to                                                     ]8;id=172566;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=485957;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/FuncErasToXOR'                       

                    INFO     Slide 'FuncErasToXOR' configuration written in                             ]8;id=632958;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=781377;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/FuncErasToXOR.json'                        

                    INFO     Slide 'FuncErasToXOR' configuration written in                             ]8;id=176067;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=995322;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/FuncErasToXOR.json'                        

## OurLB

In [60]:
%%manim -ql OurLB

class OurLB(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Simulating one XOR query takes $\Omega(\sqrt{N})$ in-place queries"), font_size=40)
        self.add(title)
        self.wait()
        self.next_slide()

        assump = Bullets(r"Imagine for sake of contradiction, $o(\sqrt{N})$ in-place queries suffice:").next_to(title, DOWN)

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )
        in_place_defn[0].stretch_to_fit_width(xor_defn[0].width)

        defns = VGroup(
            xor_defn, 
            in_place_defn
        ).scale(0.7).arrange(DOWN).next_to(assump, DOWN*4).align_to(title, LEFT)
        self.play(FadeIn(defns))
        self.next_slide()

        self.play(FadeIn(assump))
        self.next_slide()

        ckts = VGroup(
            OracleCircuit(r"P_f", r"|\psi\rangle", r"S_f|\psi\rangle"),
            OracleCircuit(r"P_f^{-1}", r"S_f|\psi\rangle", r"|\psi\rangle", flipped=True),
            OracleCircuit(r"P_f^{-1}", r"|\psi\rangle", r"S_f^{-1}|\psi\rangle", flipped=True),
            OracleCircuit(r"P_f^{-1}", r"|\psi\rangle", r"S_f|\psi\rangle", flipped=True),
            OracleCircuit(r"P_f^{-1}", r"|x\rangle|0\rangle", r"|x\rangle|f(x)\rangle", flipped=True),
            OracleCircuit(r"P_f", r"|x\rangle|0\rangle", r"|x\rangle|f^{-1}(x)\rangle", flipped=True),
        ).scale(0.7).next_to(assump, DOWN*1.5).align_to(title, RIGHT).shift(RIGHT*0.3)
        self.play(FadeIn(ckts[0][0]), FadeIn(ckts[0][1]))
        self.next_slide()
        self.play(Succession([FadeIn(gate) for gate in ckts[0][-1]], run_time=1.2), FadeIn(ckts[0][2], run_time=1.2))
        self.next_slide()

        FlipTransform(self, ckts[0], ckts[1])
        self.next_slide()
        for i in range(1,len(ckts) - 1):
            self.play(ReplacementTransform(ckts[i], ckts[i+1]))
            self.next_slide()

        contradiction = Bullets(r"Violates $\Omega(\sqrt{N})$ lower bound for computing $f^{-1}$ with in-place queries [FK18]").shift(DOWN*3)
        self.play(FadeIn(contradiction))
        self.next_slide()

Manim Community v0.19.0

[09/09/25 16:46:12] INFO     Animation 0 : Partial movie file written in                   ]8;id=278114;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=977999;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurLB/931832650_722531385_20634210                         
                             45.mp4'                                                                               

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=942856;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=250648;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurLB/1905288516_1443166991_363613                         
                             6150.mp4'                                                                             

[09/09/25 16:46:14] INFO     Animation 2 : Partial movie file written in                   ]8;id=708917;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=337269;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurLB/1905288516_2538880932_499055                         
                             83.mp4'                                                                               

[09/09/25 16:46:15] INFO     Animation 3 : Partial movie file written in                   ]8;id=851054;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=610054;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurLB/1905288516_1443166991_384670                         
                             0913.mp4'                                                                             

...

                    INFO     Rendered OurLB                                                            ]8;id=680496;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=980491;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 23 animations                                                                  

[09/09/25 16:46:42] INFO     Generated 11 slides to                                                     ]8;id=359872;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=35024;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurLB'                               

[09/09/25 16:46:42] INFO     Generated 11 slides to                                                     ]8;id=280262;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=990043;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurLB'                               

                    INFO     Slide 'OurLB' configuration written in                                     ]8;id=200619;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=120404;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurLB.json'                                

                    INFO     Slide 'OurLB' configuration written in                                     ]8;id=16550;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=16327;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurLB.json'                                

# Search With In-Place Queries

## OurAlg

In [61]:
%%manim -ql OurAlg

QUICK_RENDER_STEPS = False

class OurAlg(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Can we mimic one iteration of Grover's search?"))
        self.add(title)
        self.wait()
        self.next_slide()

        tech_title = Title(colorcode(r"Can we amplify $|x^*\rangle$ by $\Omega\left( \tfrac{1}{\sqrt{N}} \right)$ with $O(1)$ in-place queries?"), font_size=36)
        self.play(Transform(title, tech_title))
        self.next_slide()

        N = 10
        t = 1
        a_t, b_t = get_HRY_amps(t)

        init_chart = BarChart(
            [a_t] + [b_t]*(N-1),
            y_range=[0, 1, 1],
            bar_colors=[IN_PLACE] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1),
            x_axis_config={"font_size": 30},
            y_length=1,
            x_length=4,
        ).shift(LEFT*3.3).scale(0.5)

        steps = Steps(scale_factor=0.5)
        steps.add_step(
            r"\alpha |x^*\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |x\rangle"
        ).next_to(title, DOWN).shift(2.5*RIGHT).play(self)

        init_chart.align_to(steps[-1], DOWN)
        self.play(DrawBorderThenFill(init_chart))
        self.next_slide()
        
        self.remove(steps[-1])
        steps.add_step(
            r"\alpha |x^*\rangle",
            r"|0\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |x\rangle",
            r"|0\rangle",
            overlap_prev=True, cause=""
        ).play_diff(self, diff=(1,4))
        steps.remove(steps[-2])

        steps.add_step(
            r"\alpha |x^*\rangle",
            r"|x^*\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |x\rangle",
            r"|x\rangle",
            cause=r"\text{CNOT}"
        ).play_diff(self)
        
        steps.add_step(
            r"\alpha |x^*\rangle",
            r"|1\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |x\rangle",
            r"|f(x)\rangle",
            cause=r"P_f"
        ).play_diff(self)
        
        steps.add_step(
            r"\alpha |x^*\rangle",
            r"|1\rangle",
            r"\otimes |1\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |x\rangle",
            r"|f(x)\rangle",
            r"\otimes |0\rangle",
            cause=r"\text{CNOT}"
        ).play_diff(self, diff=(2,6))

        split_sep = 1.7
        mark_chart_a = init_chart.copy()
        mark_chart_b = init_chart.copy()
        mark_chart_a.change_bar_values([a_t] + [0]*(N-1))
        mark_chart_b.change_bar_values([0] + [b_t]*(N-1))
        arrow_start = init_chart.get_bottom()
        l_arrow_end = mark_chart_a.copy().shift(LEFT*split_sep).align_to(steps[-1], DOWN).get_top()
        r_arrow_end = mark_chart_a.copy().shift(RIGHT*split_sep).align_to(steps[-1], DOWN).get_top()
        l_arrow = StandardArrow(start=arrow_start, end=l_arrow_end, buff=0.5)
        r_arrow = StandardArrow(start=arrow_start, end=r_arrow_end, buff=0.5)
        self.play(
            mark_chart_a.animate.shift(LEFT*split_sep).align_to(steps[-1], DOWN),
            mark_chart_b.animate.shift(RIGHT*split_sep).align_to(steps[-1], DOWN),
            GrowArrow(l_arrow),
            GrowArrow(r_arrow),
        )
        mark_chart_a_flag = MathTex(r"\otimes |1\rangle\ ", r"+").scale(0.5).next_to(mark_chart_a)
        mark_chart_b_flag = MathTex(r"\otimes |0\rangle").scale(0.5).next_to(mark_chart_b)
        self.play(
            FadeIn(mark_chart_a_flag),
            FadeIn(mark_chart_b_flag),
        )
        self.next_slide()

        mark_label = Tex(
            "``Mark''"
        ).scale(BULLET_TEX_SCALE).move_to(VGroup(l_arrow, r_arrow).get_center_of_mass())
        self.play(FadeIn(mark_label))
        self.next_slide()
        
        steps.add_step(
            r"\alpha |x^*\rangle",
            r"|1\rangle",
            r"\otimes |1\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |f(x)\rangle",
            r"|f(x)\rangle",
            r"\otimes |0\rangle",
            cause=r"\text{ctrl-}P_f"
        ).play_diff(self)
        
        steps.add_step(
            r"\alpha |x^*\rangle",
            r"|0\rangle",
            r"\otimes |1\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |f(x)\rangle",
            r"|0\rangle",
            r"\otimes |0\rangle",
            cause=r"\text{CNOT}"
        ).play_diff(self)

        shift_charts = VGroup(
            mark_chart_a.copy(), 
            mark_chart_a_flag.copy(),
            mark_chart_b.copy(),
            mark_chart_b_flag.copy(),
        )

        l_arrow_start = mark_chart_a.get_bottom()
        r_arrow_start = mark_chart_b.get_bottom()
        l_arrow_end = shift_charts[0].copy().align_to(steps[-1], DOWN).get_top()
        r_arrow_end = shift_charts[2].copy().align_to(steps[-1], DOWN).get_top()
        l_arrow = StandardArrow(start=l_arrow_start, end=l_arrow_end, buff=0.2)
        r_arrow = StandardArrow(start=r_arrow_start, end=r_arrow_end, buff=0.2)
        self.play(
            shift_charts.animate.align_to(steps[-1], DOWN),
            GrowArrow(l_arrow),
            GrowArrow(r_arrow),
        )
        self.next_slide()

        self.play(Indicate(shift_charts[2:]))
        ShiftBars(self, shift_charts[2])
        self.next_slide()

        shift_label = Tex(
            "``Shift''"
        ).scale(BULLET_TEX_SCALE).move_to(VGroup(l_arrow, r_arrow).get_center())
        self.play(FadeIn(shift_label))
        self.next_slide()

        # ================================================================
        #  Use -n 39 flag to render from first animation after this point
        # ================================================================

        old_copies = [title, *shift_charts]
        title = title.copy()
        shift_charts = shift_charts.copy()
        for mob in old_copies:
            self.remove(mob)
        to_remove = self.mobjects
        self.add(title)
        self.play(
            *[FadeOut(mob) for mob in to_remove],
            shift_charts.animate.next_to(title, DOWN),
        )
        for mob in to_remove:
            self.remove(mob)
        self.next_slide()

        line_scale = 0.8
        tex_scale = 0.9

        betw = Tex(
            "Let's group terms by the state of the flag in the Hadamard basis!"
        ).scale(BULLET_TEX_SCALE*tex_scale).move_to(shift_charts[1][1]).shift(DOWN*line_scale)
        self.play(FadeIn(betw))
        self.next_slide()

        self.play(
            shift_charts[0].animate.shift(LEFT*1.5),
            shift_charts[1][0].animate.shift(LEFT*1.5),
            shift_charts[2:].animate.shift(RIGHT*1.5),
        )

        norm_a = MathTex(r"\frac{1}{\sqrt{2}}").scale(0.5).next_to(shift_charts[0], LEFT)
        norm_b = MathTex(r"\frac{1}{\sqrt{2}}").scale(0.5).next_to(shift_charts[2], LEFT)
        flag_a = MathTex(r"\otimes \left( |+\rangle - |-\rangle \right)").scale(0.5).next_to(shift_charts[0])
        flag_b = MathTex(r"\otimes \left( |+\rangle + |-\rangle \right)").scale(0.5).next_to(shift_charts[2])

        self.play(
            Transform(shift_charts[1][0], flag_a),
            Transform(shift_charts[3], flag_b),
            FadeIn(norm_a),
            FadeIn(norm_b),
        )
        self.next_slide()

        ref_pos = VGroup(
            shift_charts[0].copy(),
            shift_charts[2].copy(),
            shift_charts[1][1].copy(),
        ).shift(DOWN*2*line_scale)

        b0 = shift_charts[2].copy()
        b1 = shift_charts[2].copy()
        norm_b0 = norm_b.copy()
        norm_b1 = norm_b.copy()
        flag_b0 = MathTex(r"\otimes |+\rangle").scale(0.5).next_to(b0)
        flag_b1 = MathTex(r"\otimes |-\rangle").scale(0.5).next_to(b1)
        plus = ref_pos[2]

        self.play(
            FadeIn(norm_b0),
            FadeIn(norm_b1),
            FadeIn(flag_b0),
            FadeIn(flag_b1),
            FadeIn(plus),
            b0.animate.move_to(ref_pos[0]),
            b1.animate.move_to(ref_pos[1]),
            norm_b0.animate.next_to(ref_pos[0], LEFT),
            norm_b1.animate.next_to(ref_pos[1], LEFT),
            flag_b0.animate.next_to(ref_pos[0]),
            flag_b1.animate.next_to(ref_pos[1])
        )
        self.next_slide()

        a0 = shift_charts[0].copy()
        a1 = shift_charts[0].copy()
        norm_a0 = norm_a.copy()
        norm_a1 = norm_a.copy()
        flag_a0 = MathTex(r"\otimes |+\rangle").scale(0.5).next_to(a0)
        flag_a1 = MathTex(r"\otimes |-\rangle").scale(0.5).next_to(a1)
        bar_a0 = a0.bars[0].copy()
        bar_a1 = a1.bars[0].copy()
        a0.change_bar_values([0]*N)
        a1.change_bar_values([0]*N)

        self.play(
            FadeIn(norm_a0),
            FadeIn(norm_a1),
            FadeIn(flag_a0),
            FadeIn(flag_a1),
            FadeIn(bar_a0),
            FadeIn(bar_a1),
            a0.animate.move_to(ref_pos[0]),
            a1.animate.move_to(ref_pos[1]),
            bar_a0.animate.next_to(b0.bars[0], UP, buff=0),
            bar_a1.animate.align_to(b1.bars[0], UL),
            norm_a0.animate.next_to(ref_pos[0], LEFT),
            norm_a1.animate.next_to(ref_pos[1], LEFT),
            flag_a0.animate.next_to(ref_pos[0]),
            flag_a1.animate.next_to(ref_pos[1])
        )
        b1.bar_colors = init_chart.bar_colors
        b1.change_bar_values([b_t - a_t] + [0] + [b_t]*(N-2))
        dups = [a0, a1, norm_a0, norm_a1, flag_a0, flag_a1, bar_a1]
        for mob in dups:
            self.remove(mob)
        self.next_slide()

        good = Tex(
            "Amplified!"
        ).scale(BULLET_TEX_SCALE*tex_scale).next_to(ref_pos[0], DOWN)
        self.play(FadeIn(good))
        self.next_slide()

        bad = Tex(
            "Dampened..."
        ).scale(BULLET_TEX_SCALE*tex_scale).next_to(ref_pos[1], DOWN)
        self.play(FadeIn(bad))
        self.next_slide()

        redemption = Tex(
            "Looks like phase query - let's Diffuse!"
        ).scale(BULLET_TEX_SCALE*tex_scale).next_to(ref_pos[1], DOWN)
        self.play(Transform(bad, redemption))
        self.next_slide()

        d = VGroup(
            b0,
            b1,
            norm_b0,
            norm_b1,
            flag_b0,
            flag_b1,
            bar_a0,
            plus,
        ).copy()
        
        self.play(
            d.animate.shift(DOWN*2*line_scale),
        )
        GroverDiffuseTransform(self, d[1])
        self.next_slide()

        norm_plus = MathTex(r"\frac{1}{2}").scale(0.5).next_to(d[0], LEFT)
        norm_minus = MathTex(r"\frac{1}{2}").scale(0.5).next_to(d[1], LEFT)
        flag_plus = MathTex(r"\otimes \left( |0\rangle + |1\rangle \right)").scale(0.5).next_to(d[0])
        flag_minus = MathTex(r"\otimes \left( |0\rangle - |1\rangle \right)").scale(0.5).next_to(d[1])

        self.play(
            Transform(d[2], norm_plus),
            Transform(d[3], norm_minus),
            Transform(d[4], flag_plus),
            Transform(d[5], flag_minus),
        )
        self.next_slide()
        
        betw2 = Tex(
            "Grouping terms by the state of the flag in the standard basis..."
        ).scale(BULLET_TEX_SCALE*tex_scale).move_to(d[7]).shift(DOWN*line_scale)
        self.play(FadeIn(betw2))
        self.next_slide()

        target = d.copy().shift(DOWN*2*line_scale)
        
        target[0].bar_colors = init_chart.bar_colors
        target[1].bar_colors = init_chart.bar_colors
        target[0].change_bar_values([b_t + a_t - (b_t + a_t)/N] + [b_t - (b_t + a_t)/N]*(N-1))
        target[1].change_bar_values([(b_t + a_t)/N] + [-b_t + (b_t + a_t)/N] + [(b_t + a_t)/N]*(N-2))
        f0 = MathTex(r"\otimes |0\rangle").scale(0.5)
        f1 = MathTex(r"\otimes |1\rangle").scale(0.5)
        l_target = (target[0] + target[2] + f0.copy().next_to(target[0]))
        r_target = (target[1] + target[3] + f1.copy().next_to(target[1]))
        l_source_1 = (d[0] + d[2] + f0.copy().next_to(d[0]) + d[6]).copy()
        l_source_2 = (d[0] + d[2] + f1.copy().next_to(d[0]) + d[6]).copy()
        r_source_1 = (d[1] + d[3] + f0.copy().next_to(d[1])).copy()
        r_source_2 = (d[1] + d[3] + f1.copy().next_to(d[1])).copy()
        self.play(
            l_source_1.animate.align_to(l_target, DL),
            r_source_1.animate.align_to(l_target, DL),
        )
        self.add(l_target)
        for mob in [l_source_1, r_source_1]:
            self.remove(mob)
        self.play(
            l_source_2.animate.align_to(r_target, DL),
            r_source_2.animate.align_to(r_target, DL),
        )
        self.add(r_target)
        for mob in [l_source_2, r_source_2]:
            self.remove(mob)
        self.play(FadeIn(target[-1])) # plus
        self.next_slide()

        good = Tex(
            "Amplified!"
        ).scale(BULLET_TEX_SCALE*tex_scale).next_to(target[0], DOWN)
        self.play(FadeIn(good))
        self.next_slide()

        bad = Tex(
            "Negligible"
        ).scale(BULLET_TEX_SCALE*tex_scale).next_to(target[1], DOWN)
        self.play(
            Create(Cross(Rectangle(height=line_scale, width=3.5).move_to(target[1]))),
            FadeIn(bad)
        )
        self.next_slide()

Manim Community v0.19.0

[09/09/25 16:46:43] INFO     Animation 0 : Partial movie file written in                   ]8;id=976384;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=278435;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurAlg/931832650_722531385_3013152                         
                             792.mp4'                                                                              

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=489720;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=232951;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurAlg/1905288516_1443166991_23373                         
                             4035.mp4'                                                                             

[09/09/25 16:46:45] INFO     Animation 2 : Partial movie file written in                   ]8;id=220461;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=374409;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurAlg/1905288516_2143397217_15545                         
                             96757.mp4'                                                                            

[09/09/25 16:46:46] INFO     Animation 3 : Partial movie file written in                   ]8;id=670156;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=765257;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurAlg/1905288516_1443166991_68354                         
                             6511.mp4'                                                                             

...

                    INFO     Rendered OurAlg                                                           ]8;id=574114;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=76420;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 76 animations                                                                  

[09/09/25 16:49:30] INFO     Generated 29 slides to                                                     ]8;id=994532;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=428537;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurAlg'                              

[09/09/25 16:49:30] INFO     Generated 29 slides to                                                     ]8;id=840341;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=603253;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurAlg'                              

                    INFO     Slide 'OurAlg' configuration written in                                    ]8;id=188874;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=731440;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurAlg.json'                               

                    INFO     Slide 'OurAlg' configuration written in                                    ]8;id=382626;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=602491;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurAlg.json'                               

## OurBars

In [62]:
%%manim -ql OurBars

class OurBars(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode("Search with In-Place Queries"))
        self.add(title)
        self.wait()
        self.next_slide()

        split_sep = 2
        line_scale = 1.8
        chart_scale = 0.6

        # Init

        N = 10
        t = 0
        a_t, b_t = get_HRY_amps(t)

        init_chart = BarChart(
            [a_t] + [b_t]*(N-1),
            y_range=[0, 1, 1],
            bar_colors=[IN_PLACE] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1),
            x_axis_config={"font_size": 30},
            y_length=1,
            x_length=4,
        ).scale(chart_scale).next_to(title, DOWN)

        self.play(FadeIn(init_chart))
        self.next_slide()

        for i in (0,1):

            # Mark

            mark_chart_a = init_chart.copy()
            mark_chart_b = init_chart.copy()
            mark_chart_a.change_bar_values([a_t] + [0]*(N-1))
            mark_chart_b.change_bar_values([0] + [b_t]*(N-1))
            arrow_start = init_chart.get_bottom()
            l_arrow_end = mark_chart_a.copy().shift(LEFT*split_sep + DOWN*line_scale).get_top()
            r_arrow_end = mark_chart_a.copy().shift(RIGHT*split_sep + DOWN*line_scale).get_top()
            l_arrow = StandardArrow(start=arrow_start, end=l_arrow_end, buff=0.5)
            r_arrow = StandardArrow(start=arrow_start, end=r_arrow_end, buff=0.5)
            mark_chart_a_flag = MathTex(r"\otimes |1\rangle\ \ \ ", r"+").scale(0.5).next_to(mark_chart_a)
            plus = mark_chart_a_flag[1].shift(LEFT*split_sep + DOWN*line_scale)
            mark_chart_a_flag = mark_chart_a_flag[0]
            mark_chart_b_flag = MathTex(r"\otimes |0\rangle").scale(0.5).next_to(mark_chart_b)
            mark_label = Tex(
                "``Mark''"
            ).scale(BULLET_TEX_SCALE).move_to(VGroup(l_arrow, r_arrow).get_center_of_mass())
            self.play(
                (mark_chart_a + mark_chart_a_flag).animate.shift(LEFT*split_sep + DOWN*line_scale),
                (mark_chart_b + mark_chart_b_flag).animate.shift(RIGHT*split_sep + DOWN*line_scale),
                GrowArrow(l_arrow),
                GrowArrow(r_arrow),
                FadeIn(mark_label),
                FadeIn(plus, lag_ratio=0.5),
            )
            self.next_slide()

            # Shift

            shift_charts = VGroup(
                mark_chart_a.copy(), 
                VGroup(mark_chart_a_flag, plus).copy(),
                mark_chart_b.copy(),
                mark_chart_b_flag.copy(),
            )

            l_arrow_start = mark_chart_a.get_bottom()
            r_arrow_start = mark_chart_b.get_bottom()
            l_arrow_end = shift_charts[0].copy().shift(DOWN*line_scale).get_top()
            r_arrow_end = shift_charts[2].copy().shift(DOWN*line_scale).get_top()
            l_arrow = StandardArrow(start=l_arrow_start, end=l_arrow_end, buff=0.2)
            r_arrow = StandardArrow(start=r_arrow_start, end=r_arrow_end, buff=0.2)
            shift_label = Tex(
                "``Shift''"
            ).scale(BULLET_TEX_SCALE).move_to(VGroup(l_arrow, r_arrow).get_center())
            self.play(
                shift_charts.animate.shift(DOWN*line_scale),
                GrowArrow(l_arrow),
                GrowArrow(r_arrow),
                FadeIn(shift_label),
            )
            ShiftBars(self, shift_charts[2])
            self.next_slide()

            # Diffuse

            t += 1
            a_t, b_t = get_HRY_amps(t)

            final_chart = BarChart(
                [a_t] + [b_t]*(N-1),
                y_range=[0, 1, 1],
                bar_colors=[IN_PLACE] + [GRAY]*(N-1),
                bar_names=[r"$x^*$"] + [""]*(N-1),
                x_axis_config={"font_size": 30},
                y_length=1,
                x_length=4,
            ).scale(chart_scale).align_to(shift_charts[0], DOWN).shift(DOWN*line_scale)

            l_arrow_start = shift_charts[0].get_bottom()
            r_arrow_start = shift_charts[2].get_bottom()
            arrow_end = final_chart.get_top()
            l_arrow = StandardArrow(start=l_arrow_start, end=arrow_end+LEFT*0.3, buff=0.2)
            r_arrow = StandardArrow(start=r_arrow_start, end=arrow_end+RIGHT*0.3, buff=0.2)
            diffuse_label = Tex(
                "``Diffuse''"
            ).scale(BULLET_TEX_SCALE).move_to(VGroup(l_arrow, r_arrow).get_center())
            self.play(
                GrowArrow(l_arrow),
                GrowArrow(r_arrow),
                FadeIn(diffuse_label),
                FadeIn(final_chart)
            )
            self.next_slide()
            
            if i == 0:

                old_copies = [title, final_chart]
                title = title.copy()
                final_chart = final_chart.copy()
                for mob in old_copies:
                    self.remove(mob)
                to_remove = self.mobjects
                self.add(title)
                self.play(
                    *[FadeOut(mob) for mob in to_remove],
                    final_chart.animate.next_to(title, DOWN),
                )
                for mob in to_remove:
                    self.remove(mob)
                
                init_chart = final_chart
                self.next_slide()

Manim Community v0.19.0

                    INFO     Animation 0 : Partial movie file written in                   ]8;id=745994;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=953403;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurBars/931832650_722531385_306988                         
                             7955.mp4'                                                                             

[09/09/25 16:49:31] INFO     Animation 1 : Partial movie file written in                   ]8;id=295731;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=41694;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurBars/1905288516_1443166991_4932                         
                             00557.mp4'                                                                            

                    INFO     Animation 2 : Partial movie file written in                   ]8;id=119895;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=523247;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurBars/1905288516_1496661929_2532                         
                             239013.mp4'                                                                           

[09/09/25 16:49:32] INFO     Animation 3 : Partial movie file written in                   ]8;id=330591;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=54554;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurBars/1905288516_1443166991_1759                         
                             273265.mp4'                                                                           

...

                    INFO     Rendered OurBars                                                          ]8;id=19782;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=515654;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 22 animations                                                                  

[09/09/25 16:50:23] INFO     Generated 9 slides to                                                      ]8;id=577481;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=375733;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurBars'                             

[09/09/25 16:50:23] INFO     Generated 9 slides to                                                      ]8;id=242888;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=407533;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurBars'                             

                    INFO     Slide 'OurBars' configuration written in                                   ]8;id=631411;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=243575;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurBars.json'                              

                    INFO     Slide 'OurBars' configuration written in                                   ]8;id=851136;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=684928;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurBars.json'                              

# Conclusion

## StaticComparisonTable

In [63]:
%%manim -ql StaticComparisonTable

class StaticComparisonTable(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Are in-place queries stronger than XOR queries?"))
        self.add(title)
        self.wait()
        self.next_slide()

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )

        perm_inv_defn = Definition(
            r"\textsc{PermutationInversion}",
            r"\text{Given $f$, find $f^{-1}(1)$}",
        )

        idx_erase_defn = Definition(
            r"\textsc{IndexErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |0\rangle |f(x)\rangle",
        )

        func_erase_defn = Definition(
            r"\textsc{FunctionErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |x\rangle |0\rangle",
        )

        idx_erase_defn[0].match_height(perm_inv_defn[0])
        func_erase_defn[0].match_height(perm_inv_defn[0])

        def ul(upper, ucite, lower, lcite):
            u = Tex(rf"$O({upper})$ [{ucite}]")
            l = Tex(rf"$\Omega({lower})$ [{lcite}]").next_to(u, DOWN).align_to(u, LEFT)
            return VGroup(u, l)

        xp_ul = ul(r"\sqrt{N}", "Gro96", r"\sqrt{N}", "Amb02")
        xi_ul = ul(r"\sqrt{N}", "Gro96", r"\sqrt{N}", "AMRR11")
        xf_ul = MathTex(rf"\Theta(1)").scale(2)
        ip_ul = ul(r"\sqrt{N}", r"H\textbf{R}Y25", r"\sqrt{N}", "FK18][BY23")
        ii_ul = MathTex(rf"\Theta(1)").scale(2)
        if_ul = ul(r"\sqrt{N}", r"H\textbf{R}Y25", r"\sqrt{N}", r"H\textbf{R}Y25")

        ip_ul[0].set_color(RED)
        if_ul.match_color(ip_ul[0])
        func_erase_defn.match_color(ip_ul[0])

        table = VGroup(
            VMobject(), idx_erase_defn, perm_inv_defn, func_erase_defn,
            xor_defn, xi_ul, xp_ul, xf_ul,
            in_place_defn, ii_ul, ip_ul, if_ul,
        )

        table.arrange_in_grid(
            col_alignments="rccc",
            row_alignments="dcc",
        )

        conj = Tex(
            rf"[FK18]: Is $O(N)$ optimal?"
        ).next_to(ip_ul[1], UP).align_to(ip_ul[1], LEFT)

        (table+conj).match_width(title).shift(UP*0.5)
        cross = Cross(conj)

        open_prob = Tex(
            colorcode(r"STILL OPEN: \\ Does any \textbf{decision problem} require more in-place queries than XOR queries?")
        ).scale(BULLET_TEX_SCALE).next_to(table, DOWN*2)

        thankyou = Tex(
            colorcode(r"Thank you!")
        ).next_to(open_prob, DOWN*2)
        
        def fadein(*x):
            self.play(*[FadeIn(item) for item in x])
            self.next_slide()

        self.add(table)
        fadein(open_prob)
        
        self.play(Write(thankyou))
        self.next_slide()

Manim Community v0.19.0

[09/09/25 16:50:25] INFO     Animation 0 : Partial movie file written in                   ]8;id=492100;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=91971;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StaticComparisonTable/931832650_72                         
                             2531385_3769187286.mp4'                                                               

[09/09/25 16:50:26] INFO     Animation 1 : Partial movie file written in                   ]8;id=413036;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=70111;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StaticComparisonTable/1905288516_1                         
                             443166991_2506270670.mp4'                                                             

[09/09/25 16:50:32] INFO     Animation 2 : Partial movie file written in                   ]8;id=132902;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=353781;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StaticComparisonTable/1905288516_2                         
                             846520304_2364056953.mp4'                                                             

[09/09/25 16:50:36] INFO     Animation 3 : Partial movie file written in                   ]8;id=160889;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=294391;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StaticComparisonTable/1905288516_1                         
                             443166991_321417547.mp4'                                                              

...

                    INFO     Rendered StaticComparisonTable                                            ]8;id=757274;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=438295;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 6 animations                                                                   

[09/09/25 16:50:43] INFO     Generated 3 slides to                                                      ]8;id=348535;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=461089;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StaticComparisonTable'               

[09/09/25 16:50:43] INFO     Generated 3 slides to                                                      ]8;id=879639;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=628850;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StaticComparisonTable'               

                    INFO     Slide 'StaticComparisonTable' configuration written in                     ]8;id=551225;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=848851;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StaticComparisonTable.json'                

                    INFO     Slide 'StaticComparisonTable' configuration written in                     ]8;id=359661;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=32834;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StaticComparisonTable.json'                

# Purgatory

## XORvsInPlace

In [64]:
%%manim -ql XORvsInPlace

class XORvsInPlace(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        
        
        xor_bullets = [
            MathTex(r"\ket{x}\ket{a} \rightarrow \ket{x}\ket{f(x)}", tex_template=myTemplate),
            Text("Any f"),
            Text("Can erase f(x)"),
            Text("Self-inverse"),
            Text("Phase version")
        ]
        inplace_bullets = [
            MathTex(r"\ket{x} \rightarrow \ket{f(x)}", tex_template=myTemplate),
            Text("Bijections"),
            Text("Can erase x")
        ]
        
        xor_group = VGroup(*xor_bullets).arrange(DOWN)
        inplace_group = VGroup(*inplace_bullets).arrange(DOWN)

        tchart = MobjectTable(
            [[xor_group, inplace_group]],
            col_labels=[Text("XOR"), Text("In-Place")],
            arrange_in_grid_config={"row_alignments":'uu'})

        self.play(
            Write(tchart.get_horizontal_lines()), 
            Write(tchart.get_vertical_lines()),
            Write(tchart.get_col_labels()))
        self.next_slide()

        for i in range(len(xor_bullets)):
            self.play(Write(xor_bullets[i]))
            self.next_slide()

            if i < len(inplace_bullets):
                self.play(Write(inplace_bullets[i]))
                self.next_slide()

Manim Community v0.19.0

[09/09/25 16:50:45] INFO     Animation 0 : Using cached data (hash :                           ]8;id=879685;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=245866;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             931832650_3837259124_223132457)                                                       

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=682304;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=125754;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1443166991_3231532510)                                                     

[09/09/25 16:50:46] INFO     Animation 2 : Using cached data (hash :                           ]8;id=550667;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=826860;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_911082328_463503842)                                                       

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=917295;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=236067;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1443166991_3934347500)                                                     

...

                    INFO     Rendered XORvsInPlace                                                     ]8;id=399638;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=410267;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 18 animations                                                                  

[09/09/25 16:50:50] INFO     Generated 9 slides to                                                      ]8;id=52737;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=636323;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/XORvsInPlace'                        

                    INFO     Generated 9 slides to                                                      ]8;id=499030;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=402852;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/XORvsInPlace'                        

                    INFO     Slide 'XORvsInPlace' configuration written in                              ]8;id=495487;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=269887;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/XORvsInPlace.json'                         

                    INFO     Slide 'XORvsInPlace' configuration written in                              ]8;id=961307;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=296087;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/XORvsInPlace.json'                         

## Results

In [65]:
%%manim -ql Results

class Results(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        ### Known Results
        title = Title("Known Results")
        self.add(title)
        self.wait()
        self.next_slide()

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )

        idx_erase_defn = Definition(
            r"\textsc{IndexErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |0\rangle |f(x)\rangle",
        )

        func_erase_defn = Definition(
            r"\textsc{FunctionErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |x\rangle |0\rangle",
        )

        perm_inv_defn = Definition(
            r"\textsc{PermutationInversion}",
            r"\text{Given $f$, find $f^{-1}(1)$}",
        )

        idx_erase_defn[0].match_height(perm_inv_defn[0])
        func_erase_defn[0].match_height(perm_inv_defn[0])

        sep = 1.5
        oracle_defns = VGroup(
            xor_defn, in_place_defn
        ).scale(BULLET_TEX_SCALE).arrange(RIGHT*sep).next_to(title, DOWN*sep)
        prob_defns = VGroup(
            func_erase_defn, idx_erase_defn, perm_inv_defn
        ).scale(BULLET_TEX_SCALE).arrange(RIGHT*sep)
        (prob_defns-func_erase_defn).next_to(oracle_defns, DOWN*sep)
        func_erase_defn.next_to(idx_erase_defn, LEFT*sep)

        self.play(FadeIn(oracle_defns), FadeIn(prob_defns - func_erase_defn)) #TODO animate sequentially

        known_bullets = Bullets(
            r".\textsc{PermutationInversion} requires $\Omega(\sqrt{N})$ queries {\color{X_O_R} [Amb02]}{\color{IN_PLACE} [FK18]}",
            r".\textsc{PermutationInversion} possible with $O(\sqrt{N})$ XOR queries {\color{X_O_R} [Amb02]}",
            r"_.{\color{IN_PLACE} [FK18]} Conjecture $\Omega(N)$ in-place queries"
            r".Simulating an in-place query requires $\Omega(\sqrt{N})$ XOR queries",
            r".\textsc{IndexErasure} requires $\Omega(\sqrt{N})$ XOR queries {\color{X_O_R} [AMRR11]}",
            # r".A controlled in-place query can be implemented with one in-place query",
        ).next_to(prob_defns, DOWN*sep).align_to(title, LEFT)


        ### Our Results
        our_results_title = Title("Our Results")
        self.play(FadeTransform(title, our_results_title))
        self.next_slide()

        # bullets = Bullets(
        #     r".\textsc{PermutationInversion} possible with $O(\sqrt{N})$ in-place queries",
        #     r".Simulating an XOR query requires $\Omega(\sqrt{N})$ in-place queries",
        #     r"_$\implies$ \textsc{FunctionErasure} requires $\Omega(\sqrt{N})$ in-place queries",
        #     # r".A controlled in-place query can be implemented with one in-place query",
        # ).next_to(prob_defns, DOWN*sep).align_to(title, LEFT)
        
        # self.play(FadeIn(bullets[0]))
        # self.next_slide()
        
        # alg_tight = Tex(r"$\leftarrow$ tight [FK18][BY23]").scale(BULLET_TEX_SCALE*0.8).next_to(bullets[0]).shift(LEFT*0.1)
        # self.play(FadeIn(alg_tight))
        # self.next_slide()

        # self.play(FadeIn(bullets[1]))
        # self.next_slide()

        self.play(FadeIn(func_erase_defn), prob_defns.animate.next_to(oracle_defns, DOWN*sep))
        # self.play(FadeIn(bullets[2]))
        # self.next_slide()

        # LBs_tight = VGroup(
        #     Brace(bullets[1:], RIGHT), 
        #     Tex(r"tight by above \\ algorithm").scale(BULLET_TEX_SCALE*0.8)
        # ).arrange(RIGHT).next_to(bullets[1]).align_to(bullets[1], UP)
        # self.play(FadeIn(LBs_tight))
        # self.next_slide()

Manim Community v0.19.0

[09/09/25 16:50:51] INFO     Animation 0 : Partial movie file written in                   ]8;id=531974;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=719309;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Results/931832650_722531385_208323                         
                             2737.mp4'                                                                             

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=709048;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=324766;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Results/1905288516_1443166991_3375                         
                             641544.mp4'                                                                           

[09/09/25 16:50:53] INFO     Animation 2 : Partial movie file written in                   ]8;id=983521;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=153497;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Results/1905288516_4189706358_3179                         
                             186112.mp4'                                                                           

[09/09/25 16:50:56] INFO     Animation 3 : Partial movie file written in                   ]8;id=588638;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=383372;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Results/1905288516_1562478123_2427                         
                             383621.mp4'                                                                           

...

                    INFO     Rendered Results                                                          ]8;id=748819;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=602417;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 6 animations                                                                   

[09/09/25 16:51:01] INFO     Generated 3 slides to                                                      ]8;id=611634;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=41523;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Results'                             

[09/09/25 16:51:01] INFO     Generated 3 slides to                                                      ]8;id=885028;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=28327;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Results'                             

                    INFO     Slide 'Results' configuration written in                                   ]8;id=430360;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=230435;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Results.json'                              

                    INFO     Slide 'Results' configuration written in                                   ]8;id=786502;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=130565;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Results.json'                              

## InvertingCircuits1

In [66]:
%%manim -ql InvertingCircuits1

class InvertingCircuits1(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        abcde_tex = r'''\Qcircuit @C=0.7em @R=0.7em @!R {
            & \gate{A} & \multigate{1}{C} & \gate{D} & \qw \\
            & \gate{B} & \ghost{C} & \gate{E} & \qw
        }'''
        abcde_ckt = MathTex(abcde_tex, tex_template=myTemplate)
        abcde_braces = CircuitBraces(abcde_ckt, r"|\psi\rangle", r"\mathcal{C}|\psi\rangle")
        abcde = VGroup(abcde_ckt, abcde_braces)

        self.play(Write(abcde))
        self.next_slide()

        inv_abcde_tex = r'''\Qcircuit @C=0.7em @R=0.7em @!R {
            & \gate{D^{-1}} & \multigate{1}{C^{-1}} & \gate{A^{-1}} & \qw \\
            & \gate{E^{-1}} & \ghost{C^{-1}} & \gate{B^{-1}} & \qw
        }'''
        inv_abcde_ckt = MathTex(inv_abcde_tex, tex_template=myTemplate)
        inv_abcde_braces = CircuitBraces(inv_abcde_ckt, r"\mathcal{C} |\psi\rangle", r"|\psi\rangle")
        inv_abcde = VGroup(inv_abcde_ckt, inv_abcde_braces)

        FlipTransform(self, abcde, inv_abcde)
        self.next_slide()

        new_inv_abcde_braces = CircuitBraces(inv_abcde_ckt, r"|\psi\rangle", r"\mathcal{C}^{-1} |\psi\rangle")
        self.play(Transform(inv_abcde_braces, new_inv_abcde_braces))
        # self.play(*[TransformMatchingShapes(inv_abcde_braces[i], new_inv_abcde_braces[i]) for i in (1,3)])
        self.next_slide()
        

Manim Community v0.19.0

[09/09/25 16:51:02] INFO     Animation 0 : Using cached data (hash :                           ]8;id=510991;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=266375;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             931832650_2367178140_223132457)                                                       

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=709529;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=813036;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/InvertingCircuits1/1905288516_1443                         
                             166991_3917998179.mp4'                                                                

[09/09/25 16:51:03] INFO     Animation 2 : Using cached data (hash :                           ]8;id=339929;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=909066;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_2507534724_2375220172)                                                     

[09/09/25 16:51:04] INFO     Animation 3 : Partial movie file written in                   ]8;id=7948;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=530878;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/InvertingCircuits1/1905288516_1243                         
                             085686_1101682277.mp4'                                                                

...

                    INFO     Rendered InvertingCircuits1                                               ]8;id=734312;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=221667;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 7 animations                                                                   

[09/09/25 16:51:06] INFO     Generated 3 slides to                                                      ]8;id=146112;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=739087;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/InvertingCircuits1'                  

[09/09/25 16:51:06] INFO     Generated 3 slides to                                                      ]8;id=894170;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=400694;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/InvertingCircuits1'                  

                    INFO     Slide 'InvertingCircuits1' configuration written in                        ]8;id=581048;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=944741;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/InvertingCircuits1.json'                   

                    INFO     Slide 'InvertingCircuits1' configuration written in                        ]8;id=34050;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=323837;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/InvertingCircuits1.json'                   

## Kashefi

In [67]:
%%manim -ql Kashefi

class Kashefi(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        ckt = OracleCircuit(r"S_f", r"|\psi\rangle", r"P_f|\psi\rangle")
        ckt_f = OracleCircuit(r"S_f", r"|x\rangle", r"|f(x)\rangle")

        ckt_flipped = OracleCircuit(r"S_f^{-1}", r"|f(x)\rangle", r"|x\rangle", flipped=True)
        flipped_ckts = [ckt_flipped]
        flipped_ckts.append(OracleCircuit(r"S_f^{-1}", r"|x\rangle", r"|f^{-1}(x)\rangle", flipped=True))
        flipped_ckts.append(OracleCircuit(r"S_f", r"|x\rangle", r"|f^{-1}(x)\rangle", flipped=True))
        
        self.play(FadeIn(ckt[0]), FadeIn(ckt[1]))
        self.next_slide()
        self.play(Succession([FadeIn(gate) for gate in ckt[-1]], run_time=1.2), FadeIn(ckt[2], run_time=1.2))
        self.next_slide()
        self.play(ReplacementTransform(ckt, ckt_f))
        self.next_slide()

        FlipTransform(self, ckt_f, ckt_flipped)
        self.next_slide()
        for i in range(len(flipped_ckts) - 1):
            self.play(ReplacementTransform(flipped_ckts[i], flipped_ckts[i+1]))
            self.next_slide()

        contradiction = Tex(r"Violates $\Omega(\sqrt{N})$ lower bound for computing $f^{-1}$ using $S_f$").next_to(flipped_ckts[-1][0], DOWN)
        self.play(Write(contradiction))
        self.next_slide()

Manim Community v0.19.0

[09/09/25 16:51:07] INFO     Animation 0 : Using cached data (hash :                           ]8;id=133355;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=370345;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             931832650_99513097_223132457)                                                         

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=698581;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=745031;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1443166991_1385157521)                                                     

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=838561;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=394607;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1335446756_1262432934)                                                     

                    INFO     Animation 3 : Partial movie file written in                   ]8;id=810470;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=711430;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Kashefi/1905288516_1443166991_2571                         
                             837971.mp4'                                                                           

...

                    INFO     Rendered Kashefi                                                          ]8;id=639683;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=899828;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 15 animations                                                                  

[09/09/25 16:51:13] INFO     Generated 7 slides to                                                      ]8;id=278322;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=710484;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Kashefi'                             

[09/09/25 16:51:13] INFO     Generated 7 slides to                                                      ]8;id=579701;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=242873;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Kashefi'                             

                    INFO     Slide 'Kashefi' configuration written in                                   ]8;id=607900;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=179930;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Kashefi.json'                              

                    INFO     Slide 'Kashefi' configuration written in                                   ]8;id=761938;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=723;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Kashefi.json'                              

## AlgCkt

In [68]:
%%manim -ql AlgCircuit

class AlgCircuit(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        # qcircuit
        qcircuit = r'''
        \Qcircuit @C=.5em @R=0.5em @!R {
         & & & & & & \mbox{\hspace{1.8em}\textit{Mark}} & & & & & & & & \mbox{\textit{Shift}} & & & & & & & & \mbox{\textit{Diffuse the Difference}} \\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \ctrl{4}  & \qw       & \qw       & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \multigate{2}{\Ppi} & \ctrl{3}  & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \multigate{2}{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \ctrl{4}  & \qw       & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ghost{\Ppi} & \qw       & \ctrl{2}  & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \ghost{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \rstick{\ket{\psi_{t}}_\regA} \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \ctrl{4}  & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ghost{\Ppi} & \qw       & \qw       & \ctrl{1}  & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \ghost{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \push{\rule{0em}{1em}}
    \inputgroupv{2}{4}{0.5em}{2em}{\ket{\psi_{t-1}}_\regA \hspace{1.3em}} \\
    	& \lstick{\ket{0}_\regC \Big\{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \qw       & \qw       & \targ     & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ctrlo{-1} & \ctrlo{1} & \ctrlo{2} & \ctrlo{3} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \gate{H}  & \ctrl{-1} & \gate{H}  & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \meter    & \cw       & \rstick{\text{\textit{Measure} }\regC} \\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \targ     & \qw       & \qw       & \multigate{2}{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \targ     & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \targ     & \qw       & \ghost{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \targ     & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \rstick{\ket{0^n}_\regB} \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \targ     & \ghost{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \qw       & \targ     & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}}
    \inputgroupv{6}{8}{0.5em}{2em}{\ket{0^n}_\regB \hspace{0.5em}}
    \gategroup{2}{29}{4}{29}{0em}{\}}
    \gategroup{6}{21}{8}{21}{0em}{\}}
    \gategroup{2}{4}{8}{10}{0.7em}{--}
    \gategroup{2}{13}{8}{18}{0.7em}{--}
    \gategroup{2}{21}{5}{25}{0.7em}{--}\\
    }
        '''
        label = MathTex(qcircuit, tex_template=myTemplate).scale(0.5)
        self.play(Write(label))
        self.wait()
        self.next_slide()

Manim Community v0.19.0

[09/09/25 16:51:28] INFO     Animation 0 : Using cached data (hash :                           ]8;id=262177;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=200724;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             931832650_3269541140_223132457)                                                       

[09/09/25 16:51:38] INFO     Animation 1 : Using cached data (hash :                           ]8;id=301191;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=773993;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_722531385_341246984)                                                       

[09/09/25 16:51:43] INFO     Animation 2 : Using cached data (hash :                           ]8;id=90126;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=93948;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1443166991_3542080620)                                                     

                    INFO     Combining to Movie file.                                      ]8;id=552859;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=911639;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

...

                    INFO     Rendered AlgCircuit                                                       ]8;id=598492;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=413;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 3 animations                                                                   

[09/09/25 16:51:43] INFO     Generated 1 slides to                                                      ]8;id=847676;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=83699;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/AlgCircuit'                          

                    INFO     Generated 1 slides to                                                      ]8;id=386712;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=480713;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/AlgCircuit'                          

                    INFO     Slide 'AlgCircuit' configuration written in                                ]8;id=86584;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=202689;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/AlgCircuit.json'                           

                    INFO     Slide 'AlgCircuit' configuration written in                                ]8;id=452846;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=802847;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/AlgCircuit.json'                           

## StepThruOurAlgBar

In [69]:
%%manim -ql StepThruOurAlgBar

class StepThruOurAlgBar(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        N = 10
        T = int(np.ceil(np.pi/2 * np.sqrt(N)))

        chart = BarChart(
            [1/np.sqrt(N)]*N,
            y_range=[-1, 1, 1],
            bar_colors=[IN_PLACE] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1),
            x_axis_config={"font_size": 36},
        )
        y_axis_label = chart.get_y_axis_label(Tex("Amplitude").rotate(90*DEGREES), edge=LEFT, direction=LEFT, buff=0)

        self.play(DrawBorderThenFill(chart), DrawBorderThenFill(y_axis_label))
        self.next_slide()

        for t in range(1,T):
            a_t,b_t = get_HRY_amps(t,N)
            values = [a_t] + [b_t]*(N-1)
            self.play(chart.animate.change_bar_values(values))
            self.next_slide()


Manim Community v0.19.0

[09/09/25 16:51:45] INFO     Animation 0 : Partial movie file written in                   ]8;id=529188;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=99052;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruOurAlgBar/931832650_960065                         
                             280_223132457.mp4'                                                                    

[09/09/25 16:51:46] INFO     Animation 1 : Partial movie file written in                   ]8;id=365858;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=368026;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruOurAlgBar/1905288516_14431                         
                             66991_2957797163.mp4'                                                                 

[09/09/25 16:51:47] INFO     Animation 2 : Partial movie file written in                   ]8;id=903467;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=169656;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruOurAlgBar/1905288516_31239                         
                             15927_1239652051.mp4'                                                                 

[09/09/25 16:51:48] INFO     Animation 3 : Partial movie file written in                   ]8;id=32169;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=267300;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruOurAlgBar/1905288516_14431                         
                             66991_283938802.mp4'                                                                  

...

                    INFO     Rendered StepThruOurAlgBar                                                ]8;id=47617;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=285071;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 10 animations                                                                  

[09/09/25 16:51:58] INFO     Generated 5 slides to                                                      ]8;id=441307;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=72149;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruOurAlgBar'                   

[09/09/25 16:51:58] INFO     Generated 5 slides to                                                      ]8;id=950243;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=334589;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruOurAlgBar'                   

                    INFO     Slide 'StepThruOurAlgBar' configuration written in                         ]8;id=257661;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=305198;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruOurAlgBar.json'                    

                    INFO     Slide 'StepThruOurAlgBar' configuration written in                         ]8;id=216605;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=352760;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruOurAlgBar.json'                    

## [OLD] Our Alg

In [70]:
%%manim -ql OurAlgOLD

QUICK_RENDER_STEPS = True

class OurAlgOLD(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Can we mimic one iteration of Grover's search?"))
        self.add(title)
        self.next_slide()

        tech_title = Title(colorcode(r"Can we amplify $|x^*\rangle$ by $\Omega\left( \tfrac{1}{\sqrt{N}} \right)$ with $O(1)$ in-place queries?"), font_size=36)
        self.play(Transform(title, tech_title))
        self.next_slide()

        # bullets = Bullets(
        #     r"Can we mimic one iteration of Grover's algorithm?",
        #     1,
        #     r"In one iteration, we need to:",
        #     r".increase amplitude on $|x^*\rangle$ by $O\left(\frac{1}{\sqrt{N}}\right)$ \\ (Recall $x^* := f^{-1}(1)$ is our marked item)",
        #     r".use a constant number of in-place queries",
        #     font_scale=0.9, align_ref=title
        # )
        ## bullets[3][1][0][20:28].set_color(IN_PLACE)
        # bullets.play(self)

        steps = Steps(scale_factor=0.5)
        steps.add_step(r"\sum_{x \in [N]} |x\rangle", r"|0\rangle").shift(2*UP+2*RIGHT)
        steps.play(self, indices=[(-1,-1,0)])
        steps.play(self, indices=[(-1,-1,1)])
        first_step = steps.prev_result()[0]
        steps.add_step(r"\sum_{x \in [N]} |x\rangle |x\rangle", cause=r"\text{CNOT}").play(self)
        steps.add_step(r"\sum_{x \in [N]}", r"|x\rangle |f(x)\rangle", cause=r"P_f").play(self)

        first_bad_step = len(steps)
        steps.add_step(r"\sum_{x \in [N]} (-1)^{f(x) = 1} |x\rangle |f(x)\rangle").play(self)
        steps.add_step(r"\sum_{x \in [N]} (-1)^{f(x) = 1} |x\rangle |0\rangle", cause=r"\text{???}").play(self)

        funcEras = MathTex(
            r"\xrightarrow[\text{Erasure}]{\text{Function}}"
        ).scale(steps.scale_factor).next_to(steps.prev_result(), LEFT).set_color(RED)
        self.play(ReplacementTransform(steps[-1][0], funcEras))
        self.next_slide()

        cross = Cross(steps[-1])
        self.play(Create(cross))
        self.next_slide()

        steps.add(cross)
        steps.play(self, indices=range(first_bad_step, len(steps)), animation=FadeOut)
        for _ in range(first_bad_step, len(steps)):
            steps.remove(steps[-1])
        self.next_slide()
        
        premark = MathTex(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|x\rangle |f(x)\rangle", 
            r" + |x^*\rangle |1\rangle"
        ).scale(steps.scale_factor).next_to(steps[-2][-1], DOWN).align_to(steps[-2][-1], LEFT)
        self.play(
            FadeOut(steps.prev_result()[0], rate_func=slow_into), 
            FadeIn(premark[0], rate_func=slow_into),
            FadeTransform(steps.prev_result()[1], premark[1]),
            TransformFromCopy(steps.prev_result()[1], premark[2]),
        )
        steps[-1].remove(steps[-1][-1])
        steps[-1].add(premark)
        self.next_slide()

        # Mark
        steps.add_step(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|x\rangle |f(x)\rangle \otimes |0\rangle", 
            r" + |x^*\rangle |1\rangle \otimes |1\rangle"
        ).play(self)

        # Shift
        steps.add_step(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|f(x)\rangle |f(x)\rangle \otimes |0\rangle", 
            r" + |x^*\rangle |1\rangle \otimes |1\rangle",
            cause = r"\text{ctrl-}P_f"
        ).play(self)

        # Cleanup
        steps.add_step(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|f(x)\rangle", 
            r"|0\rangle", 
            r"\otimes |0\rangle + |x^*\rangle", 
            r"|0\rangle", 
            r"\otimes |1\rangle",
            cause = r"\text{CNOT}"
        ).play(self)
        clean = steps.prev_result()

        cleaner = MathTex(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|f(x)\rangle", 
            r"\otimes |0\rangle + |x^*\rangle", 
            r"\otimes |1\rangle",
        ).scale(steps.scale_factor).align_to(clean, UP).align_to(clean, LEFT)

        cleanest = MathTex(
            r"\sum_{\substack{x \in [N] \\ x \ne 1}}",
            r"|x\rangle", 
            r"\otimes |0\rangle + |x^*\rangle", 
            r"\otimes |1\rangle",
        ).scale(steps.scale_factor).align_to(clean, UP).align_to(clean, LEFT)

        self.play(
            FadeOut(clean[2]),
            FadeOut(clean[4])
        )
        clean.remove(clean[4])
        clean.remove(clean[2])
        self.play(
            TransformMatchingTex(clean, cleaner)
        )
        self.next_slide()
        for i in range(len(clean)):
            self.remove(clean[i])
        self.remove(clean)

        self.play(
            FadeTransform(cleaner[0], cleanest[0]),
            *[Transform(cleaner[i], cleanest[i]) for i in range(1,4)]
        )
        self.next_slide()

        steps[-1][-1] = cleanest

        marknshift = MathTex(
            r"\sum_{x \in [N]} |x\rangle",
            r"\rightarrow",
            r"\sum_{\substack{x \in [N] \\ x \ne 1}}",
            r"|x\rangle", 
            r"\otimes |0\rangle + |x^*\rangle", 
            r"\otimes |1\rangle",
        )

        for i in range(1,4):
            self.remove(cleaner[i])

        self.play(
            FadeOut(steps),
            TransformFromCopy(first_step, marknshift[0]),
            TransformFromCopy(steps[-1][-1], marknshift[2:]),
        )
        self.play(FadeIn(marknshift[1]))
        self.next_slide()

Manim Community v0.19.0

[09/09/25 16:51:59] INFO     Animation 0 : Using cached data (hash :                           ]8;id=304617;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=608363;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             931832650_1797274779_1931699834)                                                      

[09/09/25 16:52:00] INFO     Animation 1 : Using cached data (hash :                           ]8;id=306374;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=284219;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1443166991_1417338472)                                                     

[09/09/25 16:52:02] INFO     Animation 2 : Using cached data (hash :                           ]8;id=987576;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=67446;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_4189449029_364355843)                                                      

[09/09/25 16:52:03] INFO     Animation 3 : Using cached data (hash :                           ]8;id=676732;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=145825;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1443166991_1532018654)                                                     

...

                    INFO     Rendered OurAlgOLD                                                        ]8;id=689613;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=701972;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 18 animations                                                                  

[09/09/25 16:52:24] INFO     Generated 8 slides to                                                      ]8;id=649265;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=392554;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurAlgOLD'                           

[09/09/25 16:52:24] INFO     Generated 8 slides to                                                      ]8;id=96063;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=642530;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurAlgOLD'                           

                    INFO     Slide 'OurAlgOLD' configuration written in                                 ]8;id=74010;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=79834;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurAlgOLD.json'                            

                    INFO     Slide 'OurAlgOLD' configuration written in                                 ]8;id=221782;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=609136;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurAlgOLD.json'                            